In [1]:
import pandas as pd
import numpy as np

# 1. Load data


In [2]:
data_path = "ChallengeXHEC23022024.xlsx"
dict_df_sheets = pd.read_excel(data_path, sheet_name=None)
df_hist_jan = dict_df_sheets["JAN24"].copy()
df_clients = dict_df_sheets["clients"].copy()
df_inter = dict_df_sheets["intervenants"].copy()

In [3]:
df_hist_jan.head()

,ID Client,ID Intervenant,Date,Heure de début,Heure de fin,Prestation
0,559475456,162858075,2024-01-01,07:15:00,07:45:00,REPAS
1,559277088,162858075,2024-01-01,07:45:00,08:30:00,TOILETTE
2,87852633,78007018,2024-01-01,07:45:00,08:30:00,TOILETTE
3,243033408,810259688,2024-01-01,07:45:00,08:15:00,TOILETTE
4,814940942,710283561,2024-01-01,07:45:00,09:20:00,TOILETTE


In [4]:
df_clients.head()

,ID Client,Latitude,Longitude
0,559475456,48.721052,1.375689
1,559277088,48.721052,1.375689
2,87852633,48.691944,1.498773
3,243033408,48.726393,1.332848
4,814940942,48.733174,1.370689


In [5]:
df_inter.head()

,ID Intervenant,Latitude,Longitude,Compétences,Permis,Véhicule personnel,Dispo / Indispo,2024-01-01 00:00:00,2024-01-02 00:00:00,2024-01-03 00:00:00,...,2024-01-22 00:00:00,2024-01-23 00:00:00,2024-01-24 00:00:00,2024-01-25 00:00:00,2024-01-26 00:00:00,2024-01-27 00:00:00,2024-01-28 00:00:00,2024-01-29 00:00:00,2024-01-30 00:00:00,2024-01-31 00:00:00
0,838320706,48.738516,1.391971,"AIDE MENAGERE, REPAS, TOILETTE",Oui,Oui,"Indispo 01/01, 30/01, 31/01",0,1,1,...,1,1,1,1,1,1,1,1,0,0
1,609468992,48.640555,1.232581,"TOILETTE, REPAS, VIE SOCIALE, AIDE MENAGERE",Oui,Oui,"Indispo Tous les mercredis + 13/01, 14/01, 27/...",1,1,0,...,1,1,0,1,1,0,0,1,1,0
2,78012267,48.729206,1.371985,"HOMMES TOUTES MAINS, JARDINAGE",Oui,Oui,Dispo le 25/01,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,818696864,48.744702,1.357921,"REPAS, AIDE MENAGERE, ACCOMPAGNEMENTS COURSES,...",Oui,Oui,Indispo tous les samedis et dimanche,1,1,1,...,1,1,1,1,1,0,0,1,1,1
4,746414886,48.769455,1.197644,"TOILETTE, REPAS, AIDE MENAGERE, ACCOMPAGNEMENT...",Oui,Oui,Indispo tous les mercredis + le 05/01,1,1,0,...,1,1,0,1,1,1,1,1,1,0


# 2. Data exploration preprocessing


In [6]:
df_hist_jan.rename(
    {
        "ID Client": "id_client",
        "ID Intervenant": "id_inter",
        "Date": "date",
        "Heure de début": "start",
        "Heure de fin": "end",
        "Prestation": "obj",
    },
    axis=1,
    inplace=True,
)

df_clients.rename(
    {"ID Client": "id_client", "Latitude": "lat_client", "Longitude": "lon_client"},
    axis=1,
    inplace=True,
)

df_inter.rename(
    {
        "ID Intervenant": "id_inter",
        "Latitude": "lat_inter",
        "Longitude": "lon_inter",
        "Compétences": "skills",
        "Permis": "license",
        "Véhicule personnel": "car",
        "Dispo / Indispo": "free",
    },
    axis=1,
    inplace=True,
)

In [7]:
df_hist_jan["date"] = pd.to_datetime(df_hist_jan["date"])
df_hist_jan["start"] = pd.to_datetime(df_hist_jan["date"])
df_hist_jan["date"] = pd.to_datetime(df_hist_jan["date"])

In [8]:
df_hist_jan

,id_client,id_inter,date,start,end,obj
0,559475456,162858075,2024-01-01,2024-01-01,07:45:00,REPAS
1,559277088,162858075,2024-01-01,2024-01-01,08:30:00,TOILETTE
2,87852633,78007018,2024-01-01,2024-01-01,08:30:00,TOILETTE
3,243033408,810259688,2024-01-01,2024-01-01,08:15:00,TOILETTE
4,814940942,710283561,2024-01-01,2024-01-01,09:20:00,TOILETTE
...,...,...,...,...,...,...
2865,559277088,710283561,2024-01-31,2024-01-31,19:15:00,REPAS
2866,714782168,810259688,2024-01-31,2024-01-31,20:00:00,REPAS
2867,559475456,710283561,2024-01-31,2024-01-31,20:00:00,TOILETTE
2868,803656603,854577575,2024-01-31,2024-01-31,19:50:00,TOILETTE


# 3. EDA


In [9]:
nb_clients_jan = len(df_hist_jan["id_client"].unique())
print(f"Number of clients in January: {nb_clients_jan}")
nb_inter_jan = len(df_hist_jan["id_inter"].unique())
print(f"Number of intervenants in January: {nb_inter_jan}")

Number of clients in January: 118
Number of intervenants in January: 24


# 4. Modeling with PuLP


See precedent version


# 5. Modeling with Google OR Tools


## 5.A. Documentation


See precedent version


## 5.B. Tests with synthetic data


### 5.B.1. With initial locations and time windows


In [10]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from functools import partial

In [11]:
def create_simple_data_model():
    """Stores the data for the problem."""
    data = {}
    data["time_matrix"] = [
        [0, 3, 6, 8, 9, 5, 6],
        [3, 0, 3, 5, 7, 4, 5],
        [6, 3, 0, 3, 5, 4, 6],
        [8, 5, 3, 0, 3, 8, 4],
        [9, 7, 5, 3, 0, 9, 3],
        [5, 4, 4, 8, 9, 0, 8],
        [6, 5, 6, 4, 3, 8, 0],
    ]
    data["time_windows"] = [
        (0, 15),  # 1
        (15, 30),  # 2
        (30, 45),  # 3
        (30, 45),  # 4
        (0, 15),  # 5
        (0, 200),  # 6
        (0, 200),  # 7
    ]
    data["num_vehicles"] = 2
    data["starts"] = [5, 6]
    data["ends"] = [5, 6]
    return data

In [12]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    time_dimension = routing.GetDimensionOrDie("Time")
    total_time = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            plan_output += (
                f"{manager.IndexToNode(index)+1}"
                f" Time({solution.Min(time_var)},{solution.Max(time_var)})"
                " -> "
            )
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += (
            f"{manager.IndexToNode(index)+1}"
            f" Time({solution.Min(time_var)},{solution.Max(time_var)})\n"
        )
        plan_output += f"Time of the route: {solution.Min(time_var)}min\n"
        print(plan_output)
        total_time += solution.Min(time_var)
    print(f"Total time of all routes: {total_time}min")

In [13]:
def main():
    """Solve the VRP with time windows from specified locations."""
    # Instantiate the data problem.
    data = create_simple_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["time_matrix"]), data["num_vehicles"], data["starts"], data["ends"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def time_callback(from_index, to_index):
        """Returns the travel time between the two nodes."""
        # Convert from routing variable Index to time matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["time_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(time_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Time Windows constraint.
    time = "Time"
    routing.AddDimension(
        transit_callback_index,
        30,  # allow waiting time
        150,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time,
    )
    time_dimension = routing.GetDimensionOrDie(time)
    # Add time window constraints for each location except start and end.
    for location_idx, time_window in enumerate(data["time_windows"]):
        if location_idx in data["starts"]:
            continue
        if location_idx in data["ends"]:
            continue
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
    # # Add time window constraints for each vehicle start node.
    # depot_idx = data["depot"]
    # for vehicle_id in range(data["num_vehicles"]):
    #     index = routing.Start(vehicle_id)
    #     time_dimension.CumulVar(index).SetRange(
    #         data["time_windows"][depot_idx][0], data["time_windows"][depot_idx][1]
    #     )

    # Instantiate route start and end times to produce feasible times.
    for i in range(data["num_vehicles"]):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i))
        )
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)

In [14]:
main()

Objective: 25
Route for vehicle 0:
6 Time(0,0) -> 1 Time(5,15) -> 2 Time(15,27) -> 3 Time(30,30) -> 6 Time(34,34)
Time of the route: 34min

Route for vehicle 1:
7 Time(0,0) -> 5 Time(3,15) -> 4 Time(30,30) -> 7 Time(34,34)
Time of the route: 34min

Total time of all routes: 68min


### 5.B.2. With initial locations, time windows, and bike/car time travel


In [15]:
def create_simple_data_model():
    """Stores the data for the problem."""
    data = {}
    data["time_matrix_car"] = [
        [0, 3, 6, 8, 9, 5, 6],
        [3, 0, 3, 5, 7, 4, 5],
        [6, 3, 0, 3, 5, 4, 6],
        [8, 5, 3, 0, 3, 8, 4],
        [9, 7, 5, 3, 0, 9, 3],
        [5, 4, 4, 8, 9, 0, 8],
        [6, 5, 6, 4, 3, 8, 0],
    ]
    data["time_matrix_bike"] = [
        [0, 5, 8, 10, 11, 7, 8],
        [5, 0, 5, 7, 9, 6, 7],
        [8, 5, 0, 5, 7, 6, 8],
        [10, 7, 5, 0, 5, 10, 6],
        [11, 9, 7, 5, 0, 11, 5],
        [7, 6, 6, 10, 11, 0, 10],
        [8, 7, 8, 6, 5, 10, 0],
    ]
    data["time_windows"] = [
        (0, 15),  # 1
        (15, 30),  # 2
        (30, 31),  # 3
        (30, 31),  # 4
        (0, 15),  # 5
        (0, 200),  # 6
        (0, 200),  # 7
    ]
    data["num_vehicles"] = 2
    data["is_car"] = [0, 1]
    data["starts"] = [5, 6]
    data["ends"] = [5, 6]
    return data

In [16]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    time_dimension = routing.GetDimensionOrDie("Time")
    total_time = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        if data["is_car"][vehicle_id] == 1:
            type_str = "car"
        else:
            type_str = "bike"
        plan_output = f"Route for vehicle {vehicle_id} ({type_str}):\n"
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            plan_output += (
                f"{manager.IndexToNode(index)+1}"
                f" Time({solution.Min(time_var)},{solution.Max(time_var)})"
                " -> "
            )
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += (
            f"{manager.IndexToNode(index)+1}"
            f" Time({solution.Min(time_var)},{solution.Max(time_var)})\n"
        )
        plan_output += f"Time of the route: {solution.Min(time_var)}min\n"
        print(plan_output)
        total_time += solution.Min(time_var)
    print(f"Total time of all routes: {total_time}min")

In [17]:
def main():
    """Solve the VRP with time windows from specified locations."""
    # Instantiate the data problem.
    data = create_simple_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["time_matrix_car"]), data["num_vehicles"], data["starts"], data["ends"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def time_callback(from_index, to_index, type):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        if type == 1:  # Car
            return data["time_matrix_car"][from_node][to_node]
        else:
            return data["time_matrix_bike"][from_node][to_node]

    car_callback = partial(time_callback, type=1)
    bike_callback = partial(time_callback, type=0)

    car_index = routing.RegisterTransitCallback(car_callback)
    bike_index = routing.RegisterTransitCallback(bike_callback)

    routing.SetArcCostEvaluatorOfVehicle(bike_index, 0)
    routing.SetArcCostEvaluatorOfVehicle(car_index, 1)

    # Add Time constraint.
    dimension_name = "Time"
    routing.AddDimensionWithVehicleTransitAndCapacity(
        [bike_index, car_index],
        100,  # no slack
        [300, 300],  # vehicle maximum travel time
        True,  # start cumul to zero
        dimension_name,
    )

    time_dimension = routing.GetDimensionOrDie(dimension_name)
    # Add time window constraints for each location except start and end.
    for location_idx, time_window in enumerate(data["time_windows"]):
        if location_idx in data["starts"]:
            continue
        if location_idx in data["ends"]:
            continue
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])

    # Instantiate route start and end times to produce feasible times.
    for i in range(data["num_vehicles"]):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i))
        )
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)

In [18]:
main()

Objective: 33
Route for vehicle 0 (bike):
6 Time(0,0) -> 1 Time(7,15) -> 2 Time(15,25) -> 3 Time(30,30) -> 6 Time(36,36)
Time of the route: 36min

Route for vehicle 1 (car):
7 Time(0,0) -> 5 Time(3,15) -> 4 Time(30,30) -> 7 Time(34,34)
Time of the route: 34min

Total time of all routes: 70min


### 5.B.3. With initial locations, time windows, bike/car time travel and task execution time


In [19]:
def create_simple_data_model():
    """Stores the data for the problem."""
    data = {}
    data["time_matrix_car"] = [
        [0, 3, 6, 8, 9, 5, 6],
        [3, 0, 3, 5, 7, 4, 5],
        [6, 3, 0, 3, 5, 4, 6],
        [8, 5, 3, 0, 3, 8, 4],
        [9, 7, 5, 3, 0, 9, 3],
        [5, 4, 4, 8, 9, 0, 8],
        [6, 5, 6, 4, 3, 8, 0],
    ]
    data["time_matrix_bike"] = [
        [0, 5, 8, 10, 11, 7, 8],
        [5, 0, 5, 7, 9, 6, 7],
        [8, 5, 0, 5, 7, 6, 8],
        [10, 7, 5, 0, 5, 10, 6],
        [11, 9, 7, 5, 0, 11, 5],
        [7, 6, 6, 10, 11, 0, 10],
        [8, 7, 8, 6, 5, 10, 0],
    ]
    data["service_times"] = [1, 2, 3, 4, 5, 0, 0]
    data["time_windows"] = [
        (0, 15),  # 1
        (15, 30),  # 2
        (30, 45),  # 3
        (30, 45),  # 4
        (0, 15),  # 5
        (0, 200),  # 6
        (0, 200),  # 7
    ]
    data["time_windows_with_service"] = [
        (
            data["time_windows"][i][0] + data["service_times"][i],
            data["time_windows"][i][1],
        )
        for i in range(len(data["service_times"]))
    ]  # Requires that time windows let the time to execute the service
    data["num_vehicles"] = 2
    data["is_car"] = [0, 1]
    data["starts"] = [5, 6]
    data["ends"] = [5, 6]
    return data

In [20]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    time_dimension = routing.GetDimensionOrDie("Time")
    total_time = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        if data["is_car"][vehicle_id] == 1:
            type_str = "car"
        else:
            type_str = "bike"
        plan_output = f"Route for vehicle {vehicle_id} ({type_str}):\n"
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            plan_output += (
                f"{manager.IndexToNode(index)+1}"
                f" Time({solution.Min(time_var)},{solution.Max(time_var)})"
                " -> "
            )
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += (
            f"{manager.IndexToNode(index)+1}"
            # f"(Slack var: {time_dimension.SlackVar(index)})"
            f" Time({solution.Min(time_var)},{solution.Max(time_var)})\n"
        )
        plan_output += f"Time of the route: {solution.Min(time_var)}min\n"
        print(plan_output)
        total_time += solution.Min(time_var)
    print(f"Total time of all routes: {total_time}min")

In [21]:
def print_detailed_solution(data, manager, routing, solution):
    """Prints detailed solution on console, including service and wait times."""
    print(f"Objective: {solution.ObjectiveValue()}")
    time_dimension = routing.GetDimensionOrDie("Time")
    total_time = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        print(f"Route for vehicle {vehicle_id}:")
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            next_index = solution.Value(routing.NextVar(index))
            next_node = manager.IndexToNode(next_index)

            service_time = data["service_times"][manager.IndexToNode(index)]
            # Arrival time at current location
            arrival_time = solution.Min(time_var) - service_time

            # End of service time = arrival time + service time at current location
            end_of_service_time = solution.Min(time_var)

            if next_index != routing.End(vehicle_id):
                next_arrival_time = solution.Min(time_dimension.CumulVar(next_index))
                if data["is_car"][vehicle_id]:
                    next_travel_time = data["time_matrix_car"][
                        manager.IndexToNode(index)
                    ][next_node]
                else:
                    next_travel_time = data["time_matrix_bike"][
                        manager.IndexToNode(index)
                    ][next_node]
                next_service_time = data["service_times"][next_node]
                departure_time = (
                    next_arrival_time - next_travel_time - next_service_time
                )

            break_time = departure_time - end_of_service_time

            if next_index == routing.End(vehicle_id):
                break_time = 0
                departure_time = end_of_service_time

            # Print detailed timing for current location
            print(
                f"Location {manager.IndexToNode(index) + 1} (Arrival: {arrival_time}, "
                f"End of Service Time: {end_of_service_time}, Break: {break_time}, "
                f"Departure: {departure_time}) -> "
            )

            index = next_index

        # Handle final location's arrival time
        final_arrival_time = solution.Min(time_dimension.CumulVar(index))
        print(
            f"Location {manager.IndexToNode(index) + 1} (Arrival: {final_arrival_time})"
        )
        total_time += final_arrival_time
        print("\n")
    print(f"Total time of all routes: {total_time}min")

In [22]:
def main():
    """Solve the VRP with time windows from specified locations."""
    # Instantiate the data problem.
    data = create_simple_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["time_matrix_car"]), data["num_vehicles"], data["starts"], data["ends"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def time_callback(from_index, to_index, type):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        service_time = data["service_times"][to_node]
        if type == 1:  # Car
            return data["time_matrix_car"][from_node][to_node] + service_time
        else:
            return data["time_matrix_bike"][from_node][to_node] + service_time

    car_callback = partial(time_callback, type=1)
    bike_callback = partial(time_callback, type=0)

    car_index = routing.RegisterTransitCallback(car_callback)
    bike_index = routing.RegisterTransitCallback(bike_callback)

    routing.SetArcCostEvaluatorOfVehicle(bike_index, 0)
    routing.SetArcCostEvaluatorOfVehicle(car_index, 1)

    # Add Time constraint.
    dimension_name = "Time"
    routing.AddDimensionWithVehicleTransitAndCapacity(
        [bike_index, car_index],
        100,  # no slack
        [300, 300],  # vehicle maximum travel time
        True,  # start cumul to zero
        dimension_name,
    )

    time_dimension = routing.GetDimensionOrDie(dimension_name)

    # Add time window constraints for each location except start and end.
    for location_idx, time_window in enumerate(data["time_windows_with_service"]):
        if location_idx in data["starts"]:
            continue
        if location_idx in data["ends"]:
            continue
        index = manager.NodeToIndex(location_idx)
        start_time_window = time_window[0]
        end_time_window = time_window[1]
        time_dimension.CumulVar(index).SetRange(start_time_window, end_time_window)

    # Instantiate route start and end times to produce feasible times.
    for i in range(data["num_vehicles"]):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i))
        )
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        # print_solution(data, manager, routing, solution)
        print_detailed_solution(data, manager, routing, solution)

In [23]:
main()

Objective: 48
Route for vehicle 0:
Location 6 (Arrival: 0, End of Service Time: 0, Break: 0, Departure: 0) -> 
Location 1 (Arrival: 7, End of Service Time: 8, Break: 2, Departure: 10) -> 
Location 2 (Arrival: 15, End of Service Time: 17, Break: 0, Departure: 17) -> 
Location 6 (Arrival: 23)


Route for vehicle 1:
Location 7 (Arrival: 0, End of Service Time: 0, Break: 0, Departure: 0) -> 
Location 5 (Arrival: 3, End of Service Time: 8, Break: 19, Departure: 27) -> 
Location 4 (Arrival: 30, End of Service Time: 34, Break: 0, Departure: 34) -> 
Location 3 (Arrival: 37, End of Service Time: 40, Break: 0, Departure: 40) -> 
Location 7 (Arrival: 46)


Total time of all routes: 69min


### 5.B.4. With initial locations, time windows, bike/car time travel, task execution time and skills


In [24]:
def create_simple_data_model():
    """Stores the data for the problem."""
    data = {}
    data["time_matrix_car"] = [
        [0, 3, 6, 8, 9, 5, 6],
        [3, 0, 3, 5, 7, 4, 5],
        [6, 3, 0, 3, 5, 4, 6],
        [8, 5, 3, 0, 3, 8, 4],
        [9, 7, 5, 3, 0, 9, 3],
        [5, 4, 4, 8, 9, 0, 8],
        [6, 5, 6, 4, 3, 8, 0],
    ]
    data["time_matrix_bike"] = [
        [0, 5, 8, 10, 11, 7, 8],
        [5, 0, 5, 7, 9, 6, 7],
        [8, 5, 0, 5, 7, 6, 8],
        [10, 7, 5, 0, 5, 10, 6],
        [11, 9, 7, 5, 0, 11, 5],
        [7, 6, 6, 10, 11, 0, 10],
        [8, 7, 8, 6, 5, 10, 0],
    ]
    data["service_times"] = [1, 2, 3, 4, 5, 0, 0]
    data["time_windows"] = [
        (0, 15),  # 1
        (15, 30),  # 2
        (30, 45),  # 3
        (30, 45),  # 4
        (0, 15),  # 5
        (0, 200),  # 6
        (0, 200),  # 7
    ]
    data["time_windows_with_service"] = [
        (
            data["time_windows"][i][0] + data["service_times"][i],
            data["time_windows"][i][1],
        )
        for i in range(len(data["service_times"]))
    ]  # Requires that time windows let the time to execute the service
    data["num_vehicles"] = 2
    data["is_car"] = [0, 1]
    data["starts"] = [5, 6]
    data["ends"] = [5, 6]

    # Add skills for each vehicle
    data["vehicle_skills"] = [["skill1", "skill2"], ["skill2"]]
    # Add skill requirements for each node
    data["node_requirements"] = [
        None,
        "skill1",
        "skill2",
        "skill1",
        "skill2",
        None,
        None,
    ]
    return data

In [25]:
def print_detailed_solution(data, manager, routing, solution):
    """Prints detailed solution on console, including service and wait times."""
    print(f"Objective: {solution.ObjectiveValue()}")
    time_dimension = routing.GetDimensionOrDie("Time")
    total_time = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        vehicle_skills = str(data["vehicle_skills"][vehicle_id])
        print(f"Route for vehicle {vehicle_id} (Skills: {vehicle_skills}):")
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            next_index = solution.Value(routing.NextVar(index))
            next_node = manager.IndexToNode(next_index)

            service_time = data["service_times"][manager.IndexToNode(index)]
            skills_required = data["node_requirements"][manager.IndexToNode(index)]
            # Arrival time at current location
            arrival_time = solution.Min(time_var) - service_time

            # End of service time = arrival time + service time at current location
            end_of_service_time = solution.Min(time_var)

            if next_index != routing.End(vehicle_id):
                next_arrival_time = solution.Min(time_dimension.CumulVar(next_index))
                if data["is_car"][vehicle_id]:
                    next_travel_time = data["time_matrix_car"][
                        manager.IndexToNode(index)
                    ][next_node]
                else:
                    next_travel_time = data["time_matrix_bike"][
                        manager.IndexToNode(index)
                    ][next_node]
                next_service_time = data["service_times"][next_node]
                departure_time = (
                    next_arrival_time - next_travel_time - next_service_time
                )

            break_time = departure_time - end_of_service_time

            if next_index == routing.End(vehicle_id):
                break_time = 0
                departure_time = end_of_service_time

            # Print detailed timing for current location
            print(
                f"Location {manager.IndexToNode(index) + 1} (Arrival: {arrival_time}, "
                f"End of Service Time (Skills required: {skills_required}): {end_of_service_time}, Break: {break_time}, "
                f"Departure: {departure_time}) -> "
            )

            index = next_index

        # Handle final location's arrival time
        final_arrival_time = solution.Min(time_dimension.CumulVar(index))
        print(
            f"Location {manager.IndexToNode(index) + 1} (Arrival: {final_arrival_time})"
        )
        total_time += final_arrival_time
        print("\n")
    print(f"Total time of all routes: {total_time}min")

In [26]:
def main():
    """Solve the VRP with time windows from specified locations."""
    # Instantiate the data problem.
    data = create_simple_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["time_matrix_car"]), data["num_vehicles"], data["starts"], data["ends"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def time_callback(from_index, to_index, type):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        service_time = data["service_times"][to_node]
        if type == 1:  # Car
            return data["time_matrix_car"][from_node][to_node] + service_time
        else:
            return data["time_matrix_bike"][from_node][to_node] + service_time

    car_callback = partial(time_callback, type=1)
    bike_callback = partial(time_callback, type=0)

    car_index = routing.RegisterTransitCallback(car_callback)
    bike_index = routing.RegisterTransitCallback(bike_callback)

    routing.SetArcCostEvaluatorOfVehicle(bike_index, 0)
    routing.SetArcCostEvaluatorOfVehicle(car_index, 1)

    # Enforce node requirements based on vehicle skills
    for node_idx, requirement in enumerate(data["node_requirements"]):
        if requirement:  # Check if the node has specific skill requirements
            allowed_vehicles = [
                vehicle_id
                for vehicle_id, skills in enumerate(data["vehicle_skills"])
                if requirement in skills
            ]
            manager_idx = manager.NodeToIndex(node_idx)
            if manager_idx >= 0:  # Ensure manager index is valid
                routing.SetAllowedVehiclesForIndex(allowed_vehicles, manager_idx)

    # Add Time constraint.
    dimension_name = "Time"
    routing.AddDimensionWithVehicleTransitAndCapacity(
        [bike_index, car_index],
        100,  # no slack
        [300, 300],  # vehicle maximum travel time
        True,  # start cumul to zero
        dimension_name,
    )

    time_dimension = routing.GetDimensionOrDie(dimension_name)

    # Add time window constraints for each location except start and end.
    for location_idx, time_window in enumerate(data["time_windows_with_service"]):
        if location_idx in data["starts"]:
            continue
        if location_idx in data["ends"]:
            continue
        index = manager.NodeToIndex(location_idx)
        start_time_window = time_window[0]
        end_time_window = time_window[1]
        time_dimension.CumulVar(index).SetRange(start_time_window, end_time_window)

    # Instantiate route start and end times to produce feasible times.
    for i in range(data["num_vehicles"]):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i))
        )
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        # print_solution(data, manager, routing, solution)
        print_detailed_solution(data, manager, routing, solution)

In [27]:
main()

Objective: 51
Route for vehicle 0 (Skills: ['skill1', 'skill2']):
Location 6 (Arrival: 0, End of Service Time (Skills required: None): 0, Break: 0, Departure: 0) -> 
Location 1 (Arrival: 7, End of Service Time (Skills required: None): 8, Break: 2, Departure: 10) -> 
Location 2 (Arrival: 15, End of Service Time (Skills required: skill1): 17, Break: 6, Departure: 23) -> 
Location 4 (Arrival: 30, End of Service Time (Skills required: skill1): 34, Break: 0, Departure: 34) -> 
Location 3 (Arrival: 39, End of Service Time (Skills required: skill2): 42, Break: 0, Departure: 42) -> 
Location 6 (Arrival: 48)


Route for vehicle 1 (Skills: ['skill2']):
Location 7 (Arrival: 0, End of Service Time (Skills required: None): 0, Break: 0, Departure: 0) -> 
Location 5 (Arrival: 3, End of Service Time (Skills required: skill2): 8, Break: 0, Departure: 8) -> 
Location 7 (Arrival: 11)


Total time of all routes: 59min


### 5.C.1. With initial locations, time windows, bike/car time travel, task execution time and skills with REAL DATA


In [28]:
c_c_dist = pd.read_csv("client_client_distance.csv")
c_l_need = pd.read_csv("client_loc_need.csv")
c_n_dist = pd.read_csv("client_nurse_distance.csv")

list_clients_1 = c_c_dist["ID Client 1"].unique()
list_clients_2 = c_c_dist["ID Client 2"].unique()
full_client_list = list(set([*list_clients_1, *list_clients_2]))
nb_full_client = len(full_client_list)

In [29]:
client_id_to_idx = {}
client_idx_to_id = {}
for k in range(len(full_client_list)):
    client_id_to_idx[full_client_list[k]] = k
    client_idx_to_id[k] = full_client_list[k]

In [30]:
def convert_duration(str_duration):
    if "hour" in str_duration:
        list = str_duration.split()
        nb_minutes = int(list[0]) * 60 + int(list[2])
        return nb_minutes
    else:
        list = str_duration.split()
        nb_minutes = int(list[0])
        return nb_minutes

In [31]:
def define_client_time_matrix(
    df, vehicle_type, client_id_to_idx=client_id_to_idx, out_value=4000
):
    list_clients_1 = df["ID Client 1"].unique()
    list_clients_2 = df["ID Client 2"].unique()
    full_list = list(set([*list_clients_1, *list_clients_2]))

    matrix = [[0 for i in range(len(full_list))] for k in range(len(full_list))]
    col_type = "Duration_" + vehicle_type
    for client_1_id in full_list:
        client_1_idx = client_id_to_idx[client_1_id]
        for client_2_id in full_list:
            client_2_idx = client_id_to_idx[client_2_id]
            if client_1_id != client_2_id:
                try:
                    travel_time = convert_duration(
                        df.loc[
                            (df["ID Client 1"] == client_1_id)
                            & (df["ID Client 2"] == client_2_id),
                            col_type,
                        ].values[0]
                    )
                    matrix[client_1_idx][client_2_idx] = travel_time
                except IndexError:
                    try:
                        travel_time = convert_duration(
                            df.loc[
                                (df["ID Client 1"] == client_2_id)
                                & (df["ID Client 2"] == client_1_id),
                                col_type,
                            ].values[0]
                        )
                        matrix[client_1_idx][client_2_idx] = travel_time
                    except IndexError:
                        matrix[client_1_idx][client_2_idx] = out_value

    return np.array(matrix)

In [32]:
client_time_matrix_car = define_client_time_matrix(c_c_dist, vehicle_type="Car")
client_time_matrix_bike = define_client_time_matrix(c_c_dist, vehicle_type="Bike")

In [33]:
list_inter_columns = c_n_dist.columns[2:]
full_inter_list = list(set([int(x.split(sep="_")[2]) for x in list_inter_columns]))

inter_id_to_idx = {}
inter_idx_to_id = {}
for k in range(len(full_inter_list)):
    inter_id_to_idx[full_inter_list[k]] = k
    inter_idx_to_id[k] = full_inter_list[k]

In [34]:
def define_inter_time_matrix(
    df,
    full_client_list,
    inter_id_to_idx=inter_id_to_idx,
    client_id_to_idx=client_id_to_idx,
    vehicle_type="Car",
    out_value=4000,
):
    list_inter_columns = df.columns[2:]
    full_inter_list = list(set([int(x.split(sep="_")[2]) for x in list_inter_columns]))

    matrix = [
        [0 for i in range(len(full_client_list))] for k in range(len(full_inter_list))
    ]

    for inter_id in full_inter_list:
        vehicle_str = "_Driving" if vehicle_type == "Car" else "_Bicycling"
        col_name = "Duration_Intervenant_" + str(inter_id) + vehicle_str
        inter_idx = inter_id_to_idx[inter_id]

        for client_id in full_client_list:
            client_idx = client_id_to_idx[client_id]
            try:
                travel_time = convert_duration(
                    df.loc[df["ID Client"] == client_id, col_name].values[0]
                )
                matrix[inter_idx][client_idx] = travel_time
            except IndexError:
                matrix[inter_idx][client_idx] = out_value

    return np.array(matrix)

In [35]:
inter_time_matrix_car = define_inter_time_matrix(
    c_n_dist, full_client_list, vehicle_type="Car"
)
inter_time_matrix_bike = define_inter_time_matrix(
    c_n_dist, full_client_list, vehicle_type="Bike"
)

In [36]:
def create_full_time_matrix(client_time_matrix, inter_time_matrix, out_value=4000):
    nb_client = client_time_matrix.shape[0]
    nb_inter = inter_time_matrix.shape[0]

    matrix = [
        [out_value for i in range(nb_client + nb_inter)]
        for k in range(nb_client + nb_inter)
    ]

    for i in range(nb_client):
        for j in range(nb_client):
            matrix[i][j] = client_time_matrix[i][j]

    for i in range(nb_inter):
        for j in range(nb_client):
            matrix[i + nb_client][j] = inter_time_matrix[i][j]
            matrix[j][i + nb_client] = matrix[i + nb_client][j]

    return np.array(matrix)

In [37]:
full_time_matrix_car = create_full_time_matrix(
    client_time_matrix_car, inter_time_matrix_car
)
full_time_matrix_bike = create_full_time_matrix(
    client_time_matrix_bike, inter_time_matrix_bike
)

In [38]:
list_skills = c_l_need["Prestation"].unique()

condition = c_l_need["Date"] == "2024-01-01"
c_l_test = c_l_need[condition]
c_l_test.drop(columns=["Unnamed: 0", "Date", "Latitude", "Longitude"], inplace=True)
c_l_test = c_l_test[:50]
c_l_test

/var/folders/95/2w4zxm7j571dt7gznpftznw80000gn/T/ipykernel_1389/1660272880.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c_l_test.drop(columns=["Unnamed: 0", "Date", "Latitude", "Longitude"], inplace=True)


,ID Client,Prestation,Durée,tranche_horaire
0,559475456,REPAS,30.0,"[7, 9]"
1,559277088,TOILETTE,45.0,"[7, 11]"
2,87852633,TOILETTE,45.0,"[7, 11]"
3,243033408,TOILETTE,30.0,"[7, 11]"
4,814940942,TOILETTE,95.0,"[7, 11]"
5,864411156,TOILETTE,30.0,"[7, 11]"
6,817127257,REPAS,30.0,"[7, 9]"
7,480337694,REPAS,30.0,"[7, 9]"
8,861502972,TOILETTE,30.0,"[7, 11]"
9,87274454,TOILETTE,60.0,"[7, 11]"


In [39]:
def create_real_data_model(
    df_need,
    full_time_matrix_car,
    full_time_matrix_bike,
    list_available_inter,
    client_id_to_idx=client_id_to_idx,
    inter_id_to_idx=inter_id_to_idx,
    nb_full_client=nb_full_client,
    list_skills=list_skills,
):
    nb_needs = len(df_need)
    nb_inter = len(list_available_inter)
    data = {}

    # Initialization
    data["time_matrix_car"] = [
        [4000 for i in range(nb_needs + nb_inter)] for k in range(nb_needs + nb_inter)
    ]
    data["time_matrix_bike"] = [
        [4000 for i in range(nb_needs + nb_inter)] for k in range(nb_needs + nb_inter)
    ]
    data["service_times"] = [0 for i in range(nb_needs + nb_inter)]
    data["time_windows"] = [(0, 1440) for i in range(nb_needs + nb_inter)]

    data["num_vehicles"] = nb_inter
    data["is_car"] = [
        1 for i in range(nb_inter)
    ]  # For the moment every inter has a car
    data["starts"] = [nb_needs + i for i in range(nb_inter)]
    data["ends"] = [nb_needs + i for i in range(nb_inter)]

    data["vehicle_skills"] = [
        list_skills for i in range(nb_inter)
    ]  # For the moment every inter has all skills possible
    data["node_requirements"] = [None for i in range(nb_needs + nb_inter)]

    # Fill with corresponding values
    for i in range(nb_needs):
        need_id_client = df_need["ID Client"][i]
        need_idx_client = client_id_to_idx[need_id_client]

        for j in range(nb_needs):
            other_id_client = df_need["ID Client"][j]
            other_idx_client = client_id_to_idx[other_id_client]

            travel_time_car = full_time_matrix_car[need_idx_client, other_idx_client]
            travel_time_bike = full_time_matrix_bike[need_idx_client, other_idx_client]
            data["time_matrix_car"][i][j] = travel_time_car
            data["time_matrix_bike"][i][j] = travel_time_bike

        for j in range(nb_inter):
            id_inter = list_available_inter[j]
            idx_inter = inter_id_to_idx[id_inter]

            travel_time_car = full_time_matrix_car[need_idx_client][
                nb_full_client + idx_inter
            ]
            travel_time_bike = full_time_matrix_bike[need_idx_client][
                nb_full_client + idx_inter
            ]

            data["time_matrix_car"][i][nb_needs + j] = travel_time_car
            data["time_matrix_car"][nb_needs + j][i] = data["time_matrix_car"][i][
                nb_needs + j
            ]
            data["time_matrix_bike"][i][nb_needs + j] = travel_time_bike
            data["time_matrix_bike"][nb_needs + j][i] = data["time_matrix_bike"][i][
                nb_needs + j
            ]

        service_time = int(df_need["Durée"][i])
        data["service_times"][i] = service_time

        time_window_temp = (
            df_need["tranche_horaire"][i]
            .replace("[", "")
            .replace("]", "")
            .replace(",", "")
            .split()
        )
        time_window_start = int(float(time_window_temp[0]) * 60)
        time_window_end = int(float(time_window_temp[1]) * 60)
        data["time_windows"][i] = (time_window_start, time_window_end)

        skills_required = df_need["Prestation"][i]
        data["node_requirements"][i] = skills_required

    for i in range(nb_needs + nb_inter):
        data["time_matrix_car"][i][i] = 0
        data["time_matrix_bike"][i][i] = 0

    data["time_windows_with_service"] = [
        (
            data["time_windows"][i][0] + data["service_times"][i],
            data["time_windows"][i][1] + data["service_times"][i],
        )
        for i in range(len(data["service_times"]))
    ]

    return data

In [40]:
data_test = create_real_data_model(
    c_l_test, full_time_matrix_car, full_time_matrix_bike, full_inter_list
)

data_test

{'time_matrix_car': [[0,
   1,
   17,
   9,
   5,
   25,
   3,
   9,
   25,
   14,
   4,
   23,
   4,
   9,
   4,
   22,
   7,
   6,
   2,
   11,
   21,
   9,
   4,
   13,
   7,
   1,
   7,
   25,
   4,
   10,
   8,
   8,
   18,
   9,
   1,
   5,
   0,
   11,
   9,
   23,
   4,
   22,
   8,
   4,
   9,
   17,
   7,
   23,
   9,
   4,
   26,
   4,
   10,
   16,
   23,
   10,
   5,
   21,
   4,
   1,
   5,
   16,
   6,
   6,
   18,
   13,
   9,
   1,
   27,
   5,
   20,
   3,
   7,
   17],
  [1,
   0,
   17,
   9,
   5,
   25,
   3,
   9,
   25,
   14,
   4,
   23,
   4,
   9,
   4,
   22,
   7,
   6,
   2,
   11,
   21,
   9,
   4,
   13,
   7,
   1,
   7,
   25,
   4,
   10,
   8,
   8,
   18,
   9,
   0,
   5,
   1,
   11,
   9,
   23,
   4,
   22,
   8,
   4,
   9,
   17,
   7,
   23,
   9,
   4,
   26,
   4,
   10,
   16,
   23,
   10,
   5,
   21,
   4,
   1,
   5,
   16,
   6,
   6,
   18,
   13,
   9,
   1,
   27,
   5,
   20,
   3,
   7,
   17],
  [17,
   17,
   0,
   22,
   18,

In [41]:
def print_detailed_solution_real_data(
    data, manager, routing, solution, inter_idx_to_id=inter_idx_to_id
):
    """Prints detailed solution on console, including service and wait times."""
    print(f"Objective: {solution.ObjectiveValue()}")
    time_dimension = routing.GetDimensionOrDie("Time")
    total_time = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        # vehicle_skills = str(data["vehicle_skills"][vehicle_id])
        inter_id = inter_idx_to_id[vehicle_id]
        # print(f"Route for vehicle {inter_id} (Skills: {vehicle_skills}):")
        print(f"Route for vehicle {inter_id}:")
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            next_index = solution.Value(routing.NextVar(index))
            next_node = manager.IndexToNode(next_index)

            service_time = data["service_times"][manager.IndexToNode(index)]
            skills_required = data["node_requirements"][manager.IndexToNode(index)]
            # Arrival time at current location
            arrival_time = solution.Min(time_var) - service_time

            # End of service time = arrival time + service time at current location
            end_of_service_time = solution.Min(time_var)

            # Initialization
            departure_time = end_of_service_time

            if next_index != routing.End(vehicle_id):
                next_arrival_time = solution.Min(time_dimension.CumulVar(next_index))
                if data["is_car"][vehicle_id]:
                    next_travel_time = data["time_matrix_car"][
                        manager.IndexToNode(index)
                    ][next_node]
                else:
                    next_travel_time = data["time_matrix_bike"][
                        manager.IndexToNode(index)
                    ][next_node]
                next_service_time = data["service_times"][next_node]
                departure_time = (
                    next_arrival_time - next_travel_time - next_service_time
                )

            break_time = departure_time - end_of_service_time

            if next_index == routing.End(vehicle_id):
                break_time = 0
                departure_time = end_of_service_time

            arrival_time_str = str(arrival_time // 60) + "h" + str(arrival_time % 60)
            end_of_service_time_str = (
                str(end_of_service_time // 60) + "h" + str(end_of_service_time % 60)
            )
            break_time_str = str(break_time // 60) + "h" + str(break_time % 60)
            departure_time_str = (
                str(departure_time // 60) + "h" + str(departure_time % 60)
            )

            # Print detailed timing for current location
            print(
                f"Location {manager.IndexToNode(index) + 1} (Arrival: {arrival_time_str}, "
                f"End of Service Time (Skills required: {skills_required}): {end_of_service_time_str}, Break: {break_time_str}, "
                f"Departure: {departure_time_str}) -> "
            )

            index = next_index

        # Handle final location's arrival time
        final_arrival_time = solution.Min(time_dimension.CumulVar(index))
        print(
            f"Location {manager.IndexToNode(index) + 1} (Arrival: {final_arrival_time})"
        )
        total_time += final_arrival_time
        print("\n")
    print(f"Total time of all routes: {total_time}min")

In [42]:
def main_real_data(
    c_l_test, full_time_matrix_car, full_time_matrix_bike, full_inter_list
):
    """Solve the VRP with time windows from specified locations."""
    # Instantiate the data problem.
    data = create_real_data_model(
        c_l_test, full_time_matrix_car, full_time_matrix_bike, full_inter_list
    )

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["time_matrix_car"]), data["num_vehicles"], data["starts"], data["ends"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def time_callback(from_index, to_index, type):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        service_time = data["service_times"][to_node]
        if type == 1:  # Car
            return data["time_matrix_car"][from_node][to_node] + service_time
        else:
            return data["time_matrix_bike"][from_node][to_node] + service_time

    car_callback = partial(time_callback, type=1)
    bike_callback = partial(time_callback, type=0)

    car_index = routing.RegisterTransitCallback(car_callback)
    bike_index = routing.RegisterTransitCallback(bike_callback)

    routing.SetArcCostEvaluatorOfVehicle(bike_index, 0)
    routing.SetArcCostEvaluatorOfVehicle(car_index, 1)

    # Enforce node requirements based on vehicle skills
    for node_idx, requirement in enumerate(data["node_requirements"]):
        if requirement:  # Check if the node has specific skill requirements
            allowed_vehicles = [
                vehicle_id
                for vehicle_id, skills in enumerate(data["vehicle_skills"])
                if requirement in skills
            ]
            manager_idx = manager.NodeToIndex(node_idx)
            if manager_idx >= 0:  # Ensure manager index is valid
                routing.SetAllowedVehiclesForIndex(allowed_vehicles, manager_idx)

    # Add Time constraint.
    dimension_name = "Time"
    routing.AddDimensionWithVehicleTransitAndCapacity(
        [car_index for i in range(data["num_vehicles"])],
        10000,  # no slack
        [10000 for i in range(data["num_vehicles"])],  # vehicle maximum travel time
        True,  # start cumul to zero
        dimension_name,
    )

    time_dimension = routing.GetDimensionOrDie(dimension_name)

    # Add time window constraints for each location except start and end.
    for location_idx, time_window in enumerate(data["time_windows_with_service"]):
        if location_idx in data["starts"]:
            continue
        if location_idx in data["ends"]:
            continue
        index = manager.NodeToIndex(location_idx)
        start_time_window = time_window[0]
        end_time_window = time_window[1]
        time_dimension.CumulVar(index).SetRange(start_time_window, end_time_window)

    # Instantiate route start and end times to produce feasible times.
    for i in range(data["num_vehicles"]):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i))
        )
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.log_search = True
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        # print_solution(data, manager, routing, solution)
        print_detailed_solution_real_data(data, manager, routing, solution)

In [43]:
main_real_data(c_l_test, full_time_matrix_car, full_time_matrix_bike, full_inter_list)

Objective: 0
Route for vehicle 1452747150:
Location 51 (Arrival: 0h0, End of Service Time (Skills required: None): 0h0, Break: 0h0, Departure: 0h0) -> 
Location 51 (Arrival: 0)


Route for vehicle 480302361:
Location 52 (Arrival: 0h0, End of Service Time (Skills required: None): 0h0, Break: 0h0, Departure: 0h0) -> 
Location 52 (Arrival: 0)


Route for vehicle 818696864:
Location 53 (Arrival: 0h0, End of Service Time (Skills required: None): 0h0, Break: 0h0, Departure: 0h0) -> 
Location 53 (Arrival: 0)


Route for vehicle 648993440:
Location 54 (Arrival: 0h0, End of Service Time (Skills required: None): 0h0, Break: 0h0, Departure: 0h0) -> 
Location 54 (Arrival: 0)


Route for vehicle 746414886:
Location 55 (Arrival: 0h0, End of Service Time (Skills required: None): 0h0, Break: 0h0, Departure: 0h0) -> 
Location 55 (Arrival: 0)


Route for vehicle 854577575:
Location 56 (Arrival: 0h0, End of Service Time (Skills required: None): 0h0, Break: 0h0, Departure: 0h0) -> 
Location 56 (Arrival: 0

I0000 00:00:1710319725.798534   10950 search.cc:282] Start search (memory used = 51.25 MB)
I0000 00:00:1710319725.801796   10950 search.cc:282] Root node processed (time = 1 ms, constraints = 569, memory used = 51.67 MB)
I0000 00:00:1710319725.813500   10950 search.cc:282] Solution #0 (0, time = 12 ms, branches = 58, failures = 1, depth = 33, memory used = 52.00 MB)
I0000 00:00:1710319725.813847   10950 search.cc:282] Finished search tree (time = 13 ms, branches = 58, failures = 35, memory used = 52.00 MB)
I0000 00:00:1710319725.814163   10950 search.cc:282] End search (time = 13 ms, branches = 58, failures = 35, memory used = 52.00 MB, speed = 4461 branches/s)


### 5.C.2. With initial locations, time windows, bike/car time travel, task execution time, skills and penalty for dropped visits with REAL DATA


In [44]:
def create_real_data_model(
    df_need,
    full_time_matrix_car,
    full_time_matrix_bike,
    list_available_inter,
    client_id_to_idx=client_id_to_idx,
    inter_id_to_idx=inter_id_to_idx,
    nb_full_client=nb_full_client,
    list_skills=list_skills,
):
    nb_needs = len(df_need)
    nb_inter = len(list_available_inter)
    data = {}

    # Initialization
    data["time_matrix_car"] = [
        [4000 for i in range(nb_needs + nb_inter)] for k in range(nb_needs + nb_inter)
    ]
    data["time_matrix_bike"] = [
        [4000 for i in range(nb_needs + nb_inter)] for k in range(nb_needs + nb_inter)
    ]
    data["service_times"] = [0 for i in range(nb_needs + nb_inter)]
    data["time_windows"] = [(0, 1440) for i in range(nb_needs + nb_inter)]

    data["num_vehicles"] = nb_inter
    data["is_car"] = [
        1 for i in range(nb_inter)
    ]  # For the moment every inter has a car
    data["starts"] = [nb_needs + i for i in range(nb_inter)]
    data["ends"] = [nb_needs + i for i in range(nb_inter)]

    data["vehicle_skills"] = [
        list_skills for i in range(nb_inter)
    ]  # For the moment every inter has all skills possible
    data["node_requirements"] = [None for i in range(nb_needs + nb_inter)]

    # Fill with corresponding values
    for i in range(nb_needs):
        need_id_client = df_need["ID Client"][i]
        need_idx_client = client_id_to_idx[need_id_client]

        for j in range(nb_needs):
            other_id_client = df_need["ID Client"][j]
            other_idx_client = client_id_to_idx[other_id_client]

            travel_time_car = full_time_matrix_car[need_idx_client, other_idx_client]
            travel_time_bike = full_time_matrix_bike[need_idx_client, other_idx_client]
            data["time_matrix_car"][i][j] = travel_time_car
            data["time_matrix_bike"][i][j] = travel_time_bike

        for j in range(nb_inter):
            id_inter = list_available_inter[j]
            idx_inter = inter_id_to_idx[id_inter]

            travel_time_car = full_time_matrix_car[need_idx_client][
                nb_full_client + idx_inter
            ]
            travel_time_bike = full_time_matrix_bike[need_idx_client][
                nb_full_client + idx_inter
            ]

            data["time_matrix_car"][i][nb_needs + j] = travel_time_car
            data["time_matrix_car"][nb_needs + j][i] = data["time_matrix_car"][i][
                nb_needs + j
            ]
            data["time_matrix_bike"][i][nb_needs + j] = travel_time_bike
            data["time_matrix_bike"][nb_needs + j][i] = data["time_matrix_bike"][i][
                nb_needs + j
            ]

        service_time = int(df_need["Durée"][i])
        data["service_times"][i] = service_time

        time_window_temp = (
            df_need["tranche_horaire"][i]
            .replace("[", "")
            .replace("]", "")
            .replace(",", "")
            .split()
        )
        time_window_start = int(float(time_window_temp[0]) * 60)
        time_window_end = int(float(time_window_temp[1]) * 60)
        data["time_windows"][i] = (time_window_start, time_window_end)

        skills_required = df_need["Prestation"][i]
        data["node_requirements"][i] = skills_required

    for i in range(nb_needs + nb_inter):
        data["time_matrix_car"][i][i] = 0
        data["time_matrix_bike"][i][i] = 0

    data["time_windows_with_service"] = [
        (
            data["time_windows"][i][0] + data["service_times"][i],
            data["time_windows"][i][1] + data["service_times"][i],
        )
        for i in range(len(data["service_times"]))
    ]

    return data

In [45]:
def print_detailed_solution_real_data(
    data, manager, routing, solution, inter_idx_to_id=inter_idx_to_id
):
    """Prints detailed solution on console, including service and wait times."""
    print(f"Objective: {solution.ObjectiveValue()}")
    time_dimension = routing.GetDimensionOrDie("Time")
    total_time = 0

    # Display dropped nodes.
    dropped_nodes = "Dropped nodes:"
    for node in range(routing.Size()):
        if routing.IsStart(node) or routing.IsEnd(node):
            continue
        if solution.Value(routing.NextVar(node)) == node:
            dropped_nodes += f" {manager.IndexToNode(node)};"
    print(dropped_nodes)

    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        # vehicle_skills = str(data["vehicle_skills"][vehicle_id])
        inter_id = inter_idx_to_id[vehicle_id]
        # print(f"Route for vehicle {inter_id} (Skills: {vehicle_skills}):")
        print(f"Route for vehicle {inter_id}:")
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            next_index = solution.Value(routing.NextVar(index))
            next_node = manager.IndexToNode(next_index)

            service_time = data["service_times"][manager.IndexToNode(index)]
            skills_required = data["node_requirements"][manager.IndexToNode(index)]
            # Arrival time at current location
            arrival_time = solution.Min(time_var) - service_time

            # End of service time = arrival time + service time at current location
            end_of_service_time = solution.Min(time_var)

            # Initialization
            departure_time = end_of_service_time

            if next_index != routing.End(vehicle_id):
                next_arrival_time = solution.Min(time_dimension.CumulVar(next_index))
                if data["is_car"][vehicle_id]:
                    next_travel_time = data["time_matrix_car"][
                        manager.IndexToNode(index)
                    ][next_node]
                else:
                    next_travel_time = data["time_matrix_bike"][
                        manager.IndexToNode(index)
                    ][next_node]
                next_service_time = data["service_times"][next_node]
                departure_time = (
                    next_arrival_time - next_travel_time - next_service_time
                )

            break_time = departure_time - end_of_service_time

            if next_index == routing.End(vehicle_id):
                break_time = 0
                departure_time = end_of_service_time

            arrival_time_str = str(arrival_time // 60) + "h" + str(arrival_time % 60)
            end_of_service_time_str = (
                str(end_of_service_time // 60) + "h" + str(end_of_service_time % 60)
            )
            break_time_str = str(break_time // 60) + "h" + str(break_time % 60)
            departure_time_str = (
                str(departure_time // 60) + "h" + str(departure_time % 60)
            )

            # Print detailed timing for current location
            print(
                f"Location {manager.IndexToNode(index) + 1} (Arrival: {arrival_time_str}, "
                f"End of Service Time (Skills required: {skills_required}): {end_of_service_time_str}, Break: {break_time_str}, "
                f"Departure: {departure_time_str}) -> "
            )

            index = next_index

        # Handle final location's arrival time
        final_arrival_time = solution.Min(time_dimension.CumulVar(index))
        print(
            f"Location {manager.IndexToNode(index) + 1} (Arrival: {final_arrival_time})"
        )
        total_time += final_arrival_time
        print("\n")
    print(f"Total time of all routes: {total_time}min")

In [46]:
def main_real_data(
    c_l_test, full_time_matrix_car, full_time_matrix_bike, full_inter_list
):
    """Solve the VRP with time windows from specified locations."""
    # Instantiate the data problem.
    data = create_real_data_model(
        c_l_test, full_time_matrix_car, full_time_matrix_bike, full_inter_list
    )

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["time_matrix_car"]), data["num_vehicles"], data["starts"], data["ends"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def time_callback(from_index, to_index, type):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        service_time = data["service_times"][to_node]
        if type == 1:  # Car
            return data["time_matrix_car"][from_node][to_node] + service_time
        else:
            return data["time_matrix_bike"][from_node][to_node] + service_time

    car_callback = partial(time_callback, type=1)
    bike_callback = partial(time_callback, type=0)

    car_index = routing.RegisterTransitCallback(car_callback)
    bike_index = routing.RegisterTransitCallback(bike_callback)

    routing.SetArcCostEvaluatorOfVehicle(bike_index, 0)
    routing.SetArcCostEvaluatorOfVehicle(car_index, 1)

    # Enforce node requirements based on vehicle skills
    for node_idx, requirement in enumerate(data["node_requirements"]):
        if requirement:  # Check if the node has specific skill requirements
            allowed_vehicles = [
                vehicle_id
                for vehicle_id, skills in enumerate(data["vehicle_skills"])
                if requirement in skills
            ]
            manager_idx = manager.NodeToIndex(node_idx)
            if manager_idx >= 0:  # Ensure manager index is valid
                routing.SetAllowedVehiclesForIndex(allowed_vehicles, manager_idx)

    # Add Time constraint.
    dimension_name = "Time"
    routing.AddDimensionWithVehicleTransitAndCapacity(
        [car_index for i in range(data["num_vehicles"])],
        10000,  # no slack
        [10000 for i in range(data["num_vehicles"])],  # vehicle maximum travel time
        True,  # start cumul to zero
        dimension_name,
    )

    time_dimension = routing.GetDimensionOrDie(dimension_name)

    # Allow to drop nodes.
    penalty = 1000
    nb_needs = len(c_l_test)
    for node in range(nb_needs):
        routing.AddDisjunction([manager.NodeToIndex(node)], penalty)

    # Add time window constraints for each location except start and end.
    for location_idx, time_window in enumerate(data["time_windows_with_service"]):
        if location_idx in data["starts"]:
            continue
        if location_idx in data["ends"]:
            continue
        index = manager.NodeToIndex(location_idx)
        start_time_window = time_window[0]
        end_time_window = time_window[1]
        time_dimension.CumulVar(index).SetRange(start_time_window, end_time_window)

    # Instantiate route start and end times to produce feasible times.
    for i in range(data["num_vehicles"]):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i))
        )
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.log_search = True
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        # print_solution(data, manager, routing, solution)
        print_detailed_solution_real_data(data, manager, routing, solution)

In [47]:
list_skills = c_l_need["Prestation"].unique()

condition = c_l_need["Date"] == "2024-01-01"
c_l_test = c_l_need[condition]
c_l_test.drop(columns=["Unnamed: 0", "Date", "Latitude", "Longitude"], inplace=True)
# c_l_test = c_l_test[:70]
c_l_test.head()

/var/folders/95/2w4zxm7j571dt7gznpftznw80000gn/T/ipykernel_1389/2604372886.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c_l_test.drop(columns=["Unnamed: 0", "Date", "Latitude", "Longitude"], inplace=True)


,ID Client,Prestation,Durée,tranche_horaire
0,559475456,REPAS,30.0,"[7, 9]"
1,559277088,TOILETTE,45.0,"[7, 11]"
2,87852633,TOILETTE,45.0,"[7, 11]"
3,243033408,TOILETTE,30.0,"[7, 11]"
4,814940942,TOILETTE,95.0,"[7, 11]"


In [48]:
main_real_data(c_l_test, full_time_matrix_car, full_time_matrix_bike, full_inter_list)

Objective: 0
Dropped nodes:
Route for vehicle 1452747150:
Location 67 (Arrival: 0h0, End of Service Time (Skills required: None): 0h0, Break: 0h0, Departure: 0h0) -> 
Location 67 (Arrival: 0)


Route for vehicle 480302361:
Location 68 (Arrival: 0h0, End of Service Time (Skills required: None): 0h0, Break: 0h0, Departure: 0h0) -> 
Location 68 (Arrival: 0)


Route for vehicle 818696864:
Location 69 (Arrival: 0h0, End of Service Time (Skills required: None): 0h0, Break: 0h0, Departure: 0h0) -> 
Location 69 (Arrival: 0)


Route for vehicle 648993440:
Location 70 (Arrival: 0h0, End of Service Time (Skills required: None): 0h0, Break: 0h0, Departure: 0h0) -> 
Location 70 (Arrival: 0)


Route for vehicle 746414886:
Location 71 (Arrival: 0h0, End of Service Time (Skills required: None): 0h0, Break: 0h0, Departure: 0h0) -> 
Location 71 (Arrival: 0)


Route for vehicle 854577575:
Location 72 (Arrival: 0h0, End of Service Time (Skills required: None): 0h0, Break: 0h0, Departure: 0h0) -> 
Location

I0000 00:00:1710319726.090177   10950 routing.cc:2532] All Unperformed Solution (66000, time = 7 ms, memory used = 53.77 MB)
I0000 00:00:1710319726.090679   10950 search.cc:282] Start search (memory used = 53.77 MB)
I0000 00:00:1710319726.091300   10950 search.cc:282] Root node processed (time = 0 ms, constraints = 863, memory used = 53.88 MB)
I0000 00:00:1710319726.106567   10950 search.cc:282] Solution #0 (0, time = 15 ms, branches = 82, failures = 0, depth = 33, memory used = 54.04 MB)
I0000 00:00:1710319726.106827   10950 search.cc:282] Finished search tree (time = 16 ms, branches = 82, failures = 34, memory used = 54.04 MB)
I0000 00:00:1710319726.107223   10950 search.cc:282] End search (time = 16 ms, branches = 82, failures = 34, memory used = 54.04 MB, speed = 5125 branches/s)


### 5.C.2. With initial locations, time windows, bike/car time travel, task execution time, penalty for dropped visits and workload capacity with REAL DATA


In [49]:
def create_real_data_model(
    df_need,
    full_time_matrix_car,
    full_time_matrix_bike,
    list_available_inter,
    client_id_to_idx=client_id_to_idx,
    inter_id_to_idx=inter_id_to_idx,
    nb_full_client=nb_full_client,
    list_skills=list_skills,
):
    nb_needs = len(df_need)
    nb_inter = len(list_available_inter)
    data = {}

    # Initialization
    data["time_matrix_car"] = [
        [4000 for i in range(nb_needs + nb_inter)] for k in range(nb_needs + nb_inter)
    ]
    data["time_matrix_bike"] = [
        [4000 for i in range(nb_needs + nb_inter)] for k in range(nb_needs + nb_inter)
    ]
    data["service_times"] = [0 for i in range(nb_needs + nb_inter)]
    data["time_windows"] = [(0, 1440) for i in range(nb_needs + nb_inter)]

    data["num_vehicles"] = nb_inter
    data["is_car"] = [
        1 for i in range(nb_inter)
    ]  # For the moment every inter has a car
    data["starts"] = [nb_needs + i for i in range(nb_inter)]
    data["ends"] = [nb_needs + i for i in range(nb_inter)]

    data["vehicle_skills"] = [
        list_skills for i in range(nb_inter)
    ]  # For the moment every inter has all skills possible
    data["node_requirements"] = [None for i in range(nb_needs + nb_inter)]

    # Fill with corresponding values
    for i in range(nb_needs):
        need_id_client = df_need["ID Client"][i]
        need_idx_client = client_id_to_idx[need_id_client]

        for j in range(nb_needs):
            other_id_client = df_need["ID Client"][j]
            other_idx_client = client_id_to_idx[other_id_client]

            travel_time_car = full_time_matrix_car[need_idx_client, other_idx_client]
            travel_time_bike = full_time_matrix_bike[need_idx_client, other_idx_client]
            data["time_matrix_car"][i][j] = travel_time_car
            data["time_matrix_bike"][i][j] = travel_time_bike

        for j in range(nb_inter):
            id_inter = list_available_inter[j]
            idx_inter = inter_id_to_idx[id_inter]

            travel_time_car = full_time_matrix_car[need_idx_client][
                nb_full_client + idx_inter
            ]
            travel_time_bike = full_time_matrix_bike[need_idx_client][
                nb_full_client + idx_inter
            ]

            data["time_matrix_car"][i][nb_needs + j] = travel_time_car
            data["time_matrix_car"][nb_needs + j][i] = data["time_matrix_car"][i][
                nb_needs + j
            ]
            data["time_matrix_bike"][i][nb_needs + j] = travel_time_bike
            data["time_matrix_bike"][nb_needs + j][i] = data["time_matrix_bike"][i][
                nb_needs + j
            ]

        service_time = int(df_need["Durée"][i])
        data["service_times"][i] = service_time

        time_window_temp = (
            df_need["tranche_horaire"][i]
            .replace("[", "")
            .replace("]", "")
            .replace(",", "")
            .split()
        )
        time_window_start = int(float(time_window_temp[0]) * 60)
        time_window_end = int(float(time_window_temp[1]) * 60)
        data["time_windows"][i] = (time_window_start, time_window_end)

        skills_required = df_need["Prestation"][i]
        data["node_requirements"][i] = skills_required

    for i in range(nb_needs + nb_inter):
        data["time_matrix_car"][i][i] = 0
        data["time_matrix_bike"][i][i] = 0

    data["time_windows_with_service"] = [
        (
            data["time_windows"][i][0] + data["service_times"][i],
            data["time_windows"][i][1] + data["service_times"][i],
        )
        for i in range(len(data["service_times"]))
    ]

    data["workload_per_node"] = [1 for i in range(nb_needs)].append(
        [0 for k in range(nb_inter)]
    )  # Useless for now but could be used to differ workload from each node
    data["workload_capacity"] = [5 for i in range(nb_inter)]

    return data

In [50]:
def print_detailed_solution_real_data(
    data, manager, routing, solution, inter_idx_to_id=inter_idx_to_id
):
    """Prints detailed solution on console, including service and wait times."""
    print(f"Objective: {solution.ObjectiveValue()}")
    time_dimension = routing.GetDimensionOrDie("Time")
    overall_total_commuting_time = 0  # Initialize overall commuting time
    total_time = 0

    # Display dropped nodes.
    dropped_nodes = "Dropped nodes:"
    for node in range(routing.Size()):
        if routing.IsStart(node) or routing.IsEnd(node):
            continue
        if solution.Value(routing.NextVar(node)) == node:
            dropped_nodes += f" {manager.IndexToNode(node)};"
    print(dropped_nodes)

    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        # vehicle_skills = str(data["vehicle_skills"][vehicle_id])
        inter_id = inter_idx_to_id[vehicle_id]
        # print(f"Route for vehicle {inter_id} (Skills: {vehicle_skills}):")

        vehicle_commuting_time = 0  # Initialize commuting time for the current vehicle

        print(f"Route for vehicle {inter_id}:")
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            next_index = solution.Value(routing.NextVar(index))
            next_node = manager.IndexToNode(next_index)

            service_time = data["service_times"][manager.IndexToNode(index)]
            skills_required = data["node_requirements"][manager.IndexToNode(index)]
            # Arrival time at current location
            arrival_time = solution.Min(time_var) - service_time

            # End of service time = arrival time + service time at current location
            end_of_service_time = solution.Min(time_var)

            # Initialization
            departure_time = end_of_service_time

            if data["is_car"][vehicle_id]:
                next_travel_time = data["time_matrix_car"][manager.IndexToNode(index)][
                    next_node
                ]
            else:
                next_travel_time = data["time_matrix_bike"][manager.IndexToNode(index)][
                    next_node
                ]
            vehicle_commuting_time += next_travel_time
            overall_total_commuting_time += next_travel_time

            if next_index != routing.End(vehicle_id):
                next_arrival_time = solution.Min(time_dimension.CumulVar(next_index))
                next_service_time = data["service_times"][next_node]
                departure_time = (
                    next_arrival_time - next_travel_time - next_service_time
                )

            break_time = departure_time - end_of_service_time

            if next_index == routing.End(vehicle_id):
                break_time = 0
                departure_time = end_of_service_time

            arrival_time_str = (
                str(arrival_time // 60)
                + "h"
                + (
                    str(arrival_time % 60)
                    if arrival_time % 60 >= 10
                    else "0" + str(arrival_time % 60)
                )
            )
            end_of_service_time_str = (
                str(end_of_service_time // 60)
                + "h"
                + (
                    str(end_of_service_time % 60)
                    if end_of_service_time % 60 >= 10
                    else "0" + str(end_of_service_time % 60)
                )
            )
            break_time_str = (
                str(break_time // 60)
                + "h"
                + (
                    str(break_time % 60)
                    if break_time % 60 >= 10
                    else "0" + str(break_time % 60)
                )
            )
            departure_time_str = (
                str(departure_time // 60)
                + "h"
                + (
                    str(departure_time % 60)
                    if departure_time % 60 >= 10
                    else "0" + str(departure_time % 60)
                )
            )

            # Print detailed timing for current location
            print(
                f"Location {manager.IndexToNode(index) + 1} (Arrival: {arrival_time_str}, "
                f"End of Service Time (Skills required: {skills_required}): {end_of_service_time_str}, Break: {break_time_str}, "
                f"Departure: {departure_time_str}) -> "
            )

            index = next_index

        # Handle final location's arrival time
        final_arrival_time = solution.Min(time_dimension.CumulVar(index))
        final_arrival_time_str = (
            str(final_arrival_time // 60)
            + "h"
            + (
                str(final_arrival_time % 60)
                if final_arrival_time % 60 >= 10
                else "0" + str(final_arrival_time % 60)
            )
        )
        print(
            f"Location {manager.IndexToNode(index) + 1} (Arrival: {final_arrival_time_str})"
        )
        total_time += final_arrival_time
        print(
            f"Total commuting time for vehicle {inter_id}: {vehicle_commuting_time} minutes"
        )
        print("\n")
    print(f"Overall total commuting time: {overall_total_commuting_time} minutes")
    # print(f"Total time of all routes: {total_time}min")

In [51]:
def main_real_data(
    c_l_test, full_time_matrix_car, full_time_matrix_bike, full_inter_list
):
    """Solve the VRP with time windows from specified locations."""
    # Instantiate the data problem.
    data = create_real_data_model(
        c_l_test, full_time_matrix_car, full_time_matrix_bike, full_inter_list
    )

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["time_matrix_car"]), data["num_vehicles"], data["starts"], data["ends"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def time_callback(from_index, to_index, type):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        service_time = data["service_times"][to_node]
        if type == 1:  # Car
            return data["time_matrix_car"][from_node][to_node] + service_time
        else:
            return data["time_matrix_bike"][from_node][to_node] + service_time

    car_callback = partial(time_callback, type=1)
    bike_callback = partial(time_callback, type=0)

    car_index = routing.RegisterTransitCallback(car_callback)
    bike_index = routing.RegisterTransitCallback(bike_callback)

    routing.SetArcCostEvaluatorOfVehicle(bike_index, 0)
    routing.SetArcCostEvaluatorOfVehicle(car_index, 1)

    # Enforce node requirements based on vehicle skills
    for node_idx, requirement in enumerate(data["node_requirements"]):
        if requirement:  # Check if the node has specific skill requirements
            allowed_vehicles = [
                vehicle_id
                for vehicle_id, skills in enumerate(data["vehicle_skills"])
                if requirement in skills
            ]
            manager_idx = manager.NodeToIndex(node_idx)
            if manager_idx >= 0:  # Ensure manager index is valid
                routing.SetAllowedVehiclesForIndex(allowed_vehicles, manager_idx)

    # Add Time constraint.
    dimension_name = "Time"
    routing.AddDimensionWithVehicleTransitAndCapacity(
        [car_index for i in range(data["num_vehicles"])],
        10000,  # no slack
        [10000 for i in range(data["num_vehicles"])],  # vehicle maximum travel time
        True,  # start cumul to zero
        dimension_name,
    )

    time_dimension = routing.GetDimensionOrDie(dimension_name)

    def workload_callback(from_index):
        return 1

    workload_index = routing.RegisterUnaryTransitCallback(workload_callback)
    routing.AddDimensionWithVehicleCapacity(
        workload_index,
        slack_max=0,  # No slack
        vehicle_capacities=data["workload_capacity"],  # Maximum "load" per vehicle
        fix_start_cumul_to_zero=True,
        name="LoadBalance",
    )

    # Allow to drop nodes.
    penalty = 1000
    nb_needs = len(c_l_test)
    for node in range(nb_needs):
        routing.AddDisjunction([manager.NodeToIndex(node)], penalty)

    # Add time window constraints for each location except start and end.
    for location_idx, time_window in enumerate(data["time_windows_with_service"]):
        if location_idx in data["starts"]:
            continue
        if location_idx in data["ends"]:
            continue
        index = manager.NodeToIndex(location_idx)
        start_time_window = time_window[0]
        end_time_window = time_window[1]
        time_dimension.CumulVar(index).SetRange(start_time_window, end_time_window)

    # Instantiate route start and end times to produce feasible times.
    for i in range(data["num_vehicles"]):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i))
        )
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    # search_parameters.log_search = True
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        # print_solution(data, manager, routing, solution)
        print_detailed_solution_real_data(data, manager, routing, solution)

In [52]:
main_real_data(c_l_test, full_time_matrix_car, full_time_matrix_bike, full_inter_list)

Objective: 0
Dropped nodes:
Route for vehicle 1452747150:
Location 67 (Arrival: 0h00, End of Service Time (Skills required: None): 0h00, Break: 0h00, Departure: 0h00) -> 
Location 67 (Arrival: 0h00)
Total commuting time for vehicle 1452747150: 0 minutes


Route for vehicle 480302361:
Location 68 (Arrival: 0h00, End of Service Time (Skills required: None): 0h00, Break: 0h00, Departure: 0h00) -> 
Location 68 (Arrival: 0h00)
Total commuting time for vehicle 480302361: 0 minutes


Route for vehicle 818696864:
Location 69 (Arrival: 0h00, End of Service Time (Skills required: None): 0h00, Break: 0h00, Departure: 0h00) -> 
Location 69 (Arrival: 0h00)
Total commuting time for vehicle 818696864: 0 minutes


Route for vehicle 648993440:
Location 70 (Arrival: 0h00, End of Service Time (Skills required: None): 0h00, Break: 0h00, Departure: 0h00) -> 
Location 70 (Arrival: 0h00)
Total commuting time for vehicle 648993440: 0 minutes


Route for vehicle 746414886:
Location 71 (Arrival: 0h00, End of Se

# 6. Data preparation for scenarios


In [53]:
data_path = "ChallengeXHEC23022024.xlsx"
dict_df_sheets = pd.read_excel(data_path, sheet_name=None)
df_hist_jan = dict_df_sheets["JAN24"].copy()
df_clients = dict_df_sheets["clients"].copy()
df_inter = dict_df_sheets["intervenants"].copy()

In [54]:
client_locations = [
    (df_clients["Latitude"][k], df_clients["Longitude"][k])
    for k in range(len(df_clients))
]

inter_locations = [
    (df_inter["Latitude"][k], df_inter["Longitude"][k]) for k in range(len(df_inter))
]

In [55]:
client_id_to_location = {}
for k in range(len(df_clients)):
    client_id_to_location[int(df_clients["ID Client"][k])] = client_locations[k]

inter_id_to_location = {}
for k in range(len(df_inter)):
    inter_id_to_location[int(df_inter["ID Intervenant"][k])] = inter_locations[k]

# 6.0. Time travel estimator


In [56]:
from sklearn.ensemble import RandomForestRegressor

In [57]:
nb_clients = len(full_client_list)
nb_inter = len(full_inter_list)
nb_loc = len(full_time_matrix_car)

travel_time_data = []

for i in range(nb_loc - 1):
    for j in range(i + 1, nb_loc):
        row = []
        if i < nb_clients:
            client_id_1 = client_idx_to_id[i]
            row.append(client_id_to_location[client_id_1][0])
            row.append(client_id_to_location[client_id_1][1])

            if j < nb_clients:
                client_id_2 = client_idx_to_id[j]
                row.append(client_id_to_location[client_id_2][0])
                row.append(client_id_to_location[client_id_2][1])
            else:
                inter_id_2 = inter_idx_to_id[j - nb_clients]
                row.append(inter_id_to_location[inter_id_2][0])
                row.append(inter_id_to_location[inter_id_2][1])

        else:
            inter_id = inter_idx_to_id[i - nb_clients]
            row.append(inter_id_to_location[inter_id][0])
            row.append(inter_id_to_location[inter_id][1])

            if j < nb_clients:
                client_id_2 = client_idx_to_id[j]
                row.append(client_id_to_location[client_id_2][0])
                row.append(client_id_to_location[client_id_2][1])
            else:
                inter_id_2 = inter_idx_to_id[j - nb_clients]
                row.append(inter_id_to_location[inter_id_2][0])
                row.append(inter_id_to_location[inter_id_2][1])

        row.append(full_time_matrix_car[i][j])
        row.append(full_time_matrix_bike[i][j])

        travel_time_data.append(row)

travel_time_df = pd.DataFrame(
    travel_time_data, columns=["x_1", "y_1", "x_2", "y_2", "car", "bike"]
)
travel_time_df

,x_1,y_1,x_2,y_2,car,bike
0,48.732218,1.363744,48.705077,1.333307,7,14
1,48.732218,1.363744,48.638658,1.517034,24,63
2,48.732218,1.363744,48.787278,1.441202,16,38
3,48.732218,1.363744,48.768307,1.403410,10,24
4,48.732218,1.363744,48.725093,1.427367,11,25
...,...,...,...,...,...,...
9586,48.758099,1.210611,48.689927,1.351054,4000,4000
9587,48.758099,1.210611,48.763226,1.241120,4000,4000
9588,48.729206,1.371985,48.689927,1.351054,4000,4000
9589,48.729206,1.371985,48.763226,1.241120,4000,4000


In [58]:
condition = travel_time_df["car"] != 4000
car_travel_time_df = travel_time_df[condition].copy()
X_car = np.array(car_travel_time_df.drop(columns=["car", "bike"]))
y_car = np.array(car_travel_time_df["car"])

car_time_travel_estimator = RandomForestRegressor()
car_time_travel_estimator.fit(X_car, y_car)

RandomForestRegressor()

In [59]:
condition = travel_time_df["bike"] != 4000
bike_travel_time_df = travel_time_df[condition].copy()
X_bike = np.array(bike_travel_time_df.drop(columns=["car", "bike"]))
y_bike = np.array(bike_travel_time_df["bike"])

bike_time_travel_estimator = RandomForestRegressor()
bike_time_travel_estimator.fit(X_bike, y_bike)

RandomForestRegressor()

## 6.1. Add new client


The goal is to create artificially a new client the following way:

- Get a random ('Prestation', 'Durée', 'Tranche horaire') from an existing client;
- Get a random location

From the random location, infere the travel time to each other node


In [60]:
client_x_coords, client_y_coords = zip(*client_locations)

# Calculate mean and standard deviation for x and y
client_mean_x, client_std_x = np.mean(client_x_coords), np.std(client_x_coords)
client_mean_y, client_std_y = np.mean(client_y_coords), np.std(client_y_coords)

In [61]:
# Number of synthetic locations to generate
num_synthetic_locations = 10

# Generate synthetic locations
client_synthetic_locations = [
    (
        np.random.normal(client_mean_x, client_std_x),
        np.random.normal(client_mean_y, client_std_y),
    )
    for _ in range(num_synthetic_locations)
]

In [62]:
def add_clients_in_time_matrix(
    client_synthetic_locations,
    full_time_matrix_car,
    full_time_matrix_bike,
    nb_og_clients,
):
    len_new_matrix = len(client_synthetic_locations) + len(full_time_matrix_car)
    new_car_matrix = [
        [4000 for k in range(len_new_matrix)] for k in range(len_new_matrix)
    ]
    new_bike_matrix = [
        [4000 for k in range(len_new_matrix)] for k in range(len_new_matrix)
    ]

    nb_synth_clients = len(client_synthetic_locations)

    for i in range(len_new_matrix - 1):
        for j in range(i + 1, len_new_matrix):
            if i < nb_og_clients:
                if j < nb_og_clients:
                    new_car_matrix[i][j] = full_time_matrix_car[i][j]
                    new_bike_matrix[i][j] = full_time_matrix_bike[i][j]
                elif nb_og_clients <= j < nb_og_clients + nb_synth_clients:
                    client_id_1 = client_idx_to_id[i]
                    client_loc_1 = client_id_to_location[client_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                else:
                    new_car_matrix[i][j] = full_time_matrix_car[i][j - nb_synth_clients]
                    new_bike_matrix[i][j] = full_time_matrix_bike[i][
                        j - nb_synth_clients
                    ]
            elif nb_og_clients <= i < nb_og_clients + nb_synth_clients:
                if j < nb_og_clients + nb_synth_clients:
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                else:
                    inter_id_1 = inter_idx_to_id[j - nb_og_clients - nb_synth_clients]
                    inter_loc_1 = inter_id_to_location[inter_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                ]
                            ]
                        )[0]
                    )
            else:
                new_car_matrix[i][j] = full_time_matrix_car[i - nb_synth_clients][
                    j - nb_synth_clients
                ]
                new_bike_matrix[i][j] = full_time_matrix_bike[i - nb_synth_clients][
                    j - nb_synth_clients
                ]

            new_car_matrix[j][i] = new_car_matrix[i][j]
            new_bike_matrix[j][i] = new_bike_matrix[i][j]

    for i in range(len_new_matrix):
        new_car_matrix[i][i] = 0
        new_bike_matrix[i][i] = 0

    return np.array(new_car_matrix), np.array(new_bike_matrix)

In [63]:
new_car_matrix, new_bike_matrix = add_clients_in_time_matrix(
    client_synthetic_locations,
    full_time_matrix_car,
    full_time_matrix_bike,
    len(full_client_list),
)

## 6.2. Add new inter


The goal is to create artificially a new inter the following way:

- Get a random ('Skills', 'Disponibility', 'Car') from existing inter (can be different ones);
- Get a random location

From the random location, infere the travel time to each other node


In [64]:
def add_clients_and_inter_in_time_matrix(
    client_synthetic_locations,
    inter_synthetic_locations,
    full_time_matrix_car,
    full_time_matrix_bike,
    nb_og_clients,
    nb_og_inter,
):
    len_new_matrix = (
        len(client_synthetic_locations)
        + len(inter_synthetic_locations)
        + len(full_time_matrix_car)
    )
    new_car_matrix = [
        [4000 for k in range(len_new_matrix)] for k in range(len_new_matrix)
    ]
    new_bike_matrix = [
        [4000 for k in range(len_new_matrix)] for k in range(len_new_matrix)
    ]

    nb_synth_clients = len(client_synthetic_locations)
    nb_synth_inter = len(inter_synthetic_locations)

    for i in range(len_new_matrix - 1):
        for j in range(i + 1, len_new_matrix):
            if i < nb_og_clients:
                if j < nb_og_clients:
                    new_car_matrix[i][j] = full_time_matrix_car[i][j]
                    new_bike_matrix[i][j] = full_time_matrix_bike[i][j]
                elif nb_og_clients <= j < nb_og_clients + nb_synth_clients:
                    client_id_1 = client_idx_to_id[i]
                    client_loc_1 = client_id_to_location[client_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                elif (
                    nb_og_clients + nb_synth_clients
                    <= j
                    < nb_og_clients + nb_synth_clients + nb_og_inter
                ):
                    new_car_matrix[i][j] = full_time_matrix_car[i][j - nb_synth_clients]
                    new_bike_matrix[i][j] = full_time_matrix_bike[i][
                        j - nb_synth_clients
                    ]
                else:
                    client_id_1 = client_idx_to_id[i]
                    client_loc_1 = client_id_to_location[client_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
            elif nb_og_clients <= i < nb_og_clients + nb_synth_clients:
                if j < nb_og_clients + nb_synth_clients:
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                elif (
                    nb_og_clients + nb_synth_clients
                    <= j
                    < nb_og_clients + nb_synth_clients + nb_og_inter
                ):
                    inter_id_1 = inter_idx_to_id[j - nb_og_clients - nb_synth_clients]
                    inter_loc_1 = inter_id_to_location[inter_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                ]
                            ]
                        )[0]
                    )
                else:
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
            elif (
                nb_og_clients + nb_synth_clients
                <= i
                < nb_og_clients + nb_synth_clients + nb_og_inter
            ):
                if j < nb_og_clients + nb_synth_clients + nb_og_inter:
                    new_car_matrix[i][j] = full_time_matrix_car[i - nb_synth_clients][
                        j - nb_synth_clients
                    ]
                    new_bike_matrix[i][j] = full_time_matrix_bike[i - nb_synth_clients][
                        j - nb_synth_clients
                    ]
                else:
                    inter_id_1 = inter_idx_to_id[i - nb_synth_clients - nb_og_clients]
                    inter_loc_1 = inter_id_to_location[inter_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
            else:
                new_car_matrix[i][j] = round(
                    car_time_travel_estimator.predict(
                        [
                            [
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][1],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][1],
                            ]
                        ]
                    )[0]
                )
                new_bike_matrix[i][j] = round(
                    bike_time_travel_estimator.predict(
                        [
                            [
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][1],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][1],
                            ]
                        ]
                    )[0]
                )

            new_car_matrix[j][i] = new_car_matrix[i][j]
            new_bike_matrix[j][i] = new_bike_matrix[i][j]

    for i in range(len_new_matrix):
        new_car_matrix[i][i] = 0
        new_bike_matrix[i][i] = 0

    return np.array(new_car_matrix), np.array(new_bike_matrix)

In [65]:
def add_inter_in_time_matrix(
    inter_synthetic_locations,
    full_time_matrix_car,
    full_time_matrix_bike,
    nb_og_clients,
    nb_og_inter,
):
    len_new_matrix = +len(inter_synthetic_locations) + len(full_time_matrix_car)
    new_car_matrix = [
        [4000 for k in range(len_new_matrix)] for k in range(len_new_matrix)
    ]
    new_bike_matrix = [
        [4000 for k in range(len_new_matrix)] for k in range(len_new_matrix)
    ]

    for i in range(len_new_matrix - 1):
        for j in range(i + 1, len_new_matrix):
            if i < nb_og_clients:
                if j < nb_og_clients:
                    new_car_matrix[i][j] = full_time_matrix_car[i][j]
                    new_bike_matrix[i][j] = full_time_matrix_bike[i][j]
                elif nb_og_clients <= j < nb_og_clients + nb_og_inter:
                    new_car_matrix[i][j] = full_time_matrix_car[i][j]
                    new_bike_matrix[i][j] = full_time_matrix_bike[i][j]
                else:
                    client_id_1 = client_idx_to_id[i]
                    client_loc_1 = client_id_to_location[client_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
            elif nb_og_clients <= i < nb_og_clients + nb_og_inter:
                if j < nb_og_clients + nb_og_inter:
                    new_car_matrix[i][j] = full_time_matrix_car[i][j]
                    new_bike_matrix[i][j] = full_time_matrix_bike[i][j]
                else:
                    inter_id_1 = inter_idx_to_id[i - nb_og_clients]
                    inter_loc_1 = inter_id_to_location[inter_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
            else:
                new_car_matrix[i][j] = round(
                    car_time_travel_estimator.predict(
                        [
                            [
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_og_inter
                                ][1],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_og_inter
                                ][1],
                            ]
                        ]
                    )[0]
                )
                new_bike_matrix[i][j] = round(
                    bike_time_travel_estimator.predict(
                        [
                            [
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_og_inter
                                ][1],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_og_inter
                                ][1],
                            ]
                        ]
                    )[0]
                )

            new_car_matrix[j][i] = new_car_matrix[i][j]
            new_bike_matrix[j][i] = new_bike_matrix[i][j]

    for i in range(len_new_matrix):
        new_car_matrix[i][i] = 0
        new_bike_matrix[i][i] = 0

    return np.array(new_car_matrix), np.array(new_bike_matrix)

In [66]:
inter_x_coords, inter_y_coords = zip(*inter_locations)

# Calculate mean and standard deviation for x and y
inter_mean_x, inter_std_x = np.mean(inter_x_coords), np.std(inter_x_coords)
inter_mean_y, inter_std_y = np.mean(inter_y_coords), np.std(inter_y_coords)

In [67]:
# Number of synthetic locations to generate
num_synthetic_locations = 5

# Generate synthetic locations
inter_synthetic_locations = [
    (
        np.random.normal(inter_mean_x, inter_std_x),
        np.random.normal(inter_mean_y, inter_std_y),
    )
    for _ in range(num_synthetic_locations)
]

In [68]:
new_car_matrix, new_bike_matrix = add_inter_in_time_matrix(
    inter_synthetic_locations, full_time_matrix_car, full_time_matrix_bike, 115, 24
)

new_car_matrix

array([[ 0,  7, 24, ...,  7, 14,  7],
       [ 7,  0, 24, ...,  3, 13,  9],
       [24, 24,  0, ..., 24, 29, 21],
       ...,
       [ 7,  3, 24, ...,  0, 12, 10],
       [14, 13, 29, ..., 12,  0, 14],
       [ 7,  9, 21, ..., 10, 14,  0]])

In [69]:
new_car_matrix, new_bike_matrix = add_clients_and_inter_in_time_matrix(
    client_synthetic_locations,
    inter_synthetic_locations,
    full_time_matrix_car,
    full_time_matrix_bike,
    115,
    24,
)

new_car_matrix

array([[ 0,  7, 24, ...,  7, 14,  7],
       [ 7,  0, 24, ...,  3, 13,  9],
       [24, 24,  0, ..., 24, 29, 21],
       ...,
       [ 7,  3, 24, ...,  0, 12, 10],
       [14, 13, 29, ..., 12,  0, 14],
       [ 7,  9, 21, ..., 10, 14,  0]])

## 6.3. Add actors refined


In [ ]:
client_x_coords, client_y_coords = zip(*client_locations)

# Calculate mean and standard deviation for x and y
client_mean_x, client_std_x = np.mean(client_x_coords), np.std(client_x_coords)
client_mean_y, client_std_y = np.mean(client_y_coords), np.std(client_y_coords)

# Number of synthetic locations to generate
num_synthetic_locations = 10

# Generate synthetic locations
client_synthetic_locations = [
    (
        np.random.normal(client_mean_x, client_std_x),
        np.random.normal(client_mean_y, client_std_y),
    )
    for _ in range(num_synthetic_locations)
]

In [ ]:
def sample_random_loc(nb_synth_loc, list_og_locations):
    og_x_coords, og_y_coords = zip(*list_og_locations)

    # Calculate mean and standard deviation for x and y
    og_mean_x, og_std_x = np.mean(og_x_coords), np.std(og_x_coords)
    og_mean_y, og_std_y = np.mean(og_y_coords), np.std(og_y_coords)

    # Generate synthetic locations
    list_synth_locations = [
        (
            np.random.normal(og_mean_x, og_std_x),
            np.random.normal(og_mean_y, og_std_y),
        )
        for _ in range(nb_synth_loc)
    ]

    return list_synth_locations

In [76]:
df_inter.head()

,ID Intervenant,Latitude,Longitude,Compétences,Permis,Véhicule personnel,Dispo / Indispo,2024-01-01 00:00:00,2024-01-02 00:00:00,2024-01-03 00:00:00,...,2024-01-22 00:00:00,2024-01-23 00:00:00,2024-01-24 00:00:00,2024-01-25 00:00:00,2024-01-26 00:00:00,2024-01-27 00:00:00,2024-01-28 00:00:00,2024-01-29 00:00:00,2024-01-30 00:00:00,2024-01-31 00:00:00
0,838320706,48.738516,1.391971,"AIDE MENAGERE, REPAS, TOILETTE",Oui,Oui,"Indispo 01/01, 30/01, 31/01",0,1,1,...,1,1,1,1,1,1,1,1,0,0
1,609468992,48.640555,1.232581,"TOILETTE, REPAS, VIE SOCIALE, AIDE MENAGERE",Oui,Oui,"Indispo Tous les mercredis + 13/01, 14/01, 27/...",1,1,0,...,1,1,0,1,1,0,0,1,1,0
2,78012267,48.729206,1.371985,"HOMMES TOUTES MAINS, JARDINAGE",Oui,Oui,Dispo le 25/01,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,818696864,48.744702,1.357921,"REPAS, AIDE MENAGERE, ACCOMPAGNEMENTS COURSES,...",Oui,Oui,Indispo tous les samedis et dimanche,1,1,1,...,1,1,1,1,1,0,0,1,1,1
4,746414886,48.769455,1.197644,"TOILETTE, REPAS, AIDE MENAGERE, ACCOMPAGNEMENT...",Oui,Oui,Indispo tous les mercredis + le 05/01,1,1,0,...,1,1,0,1,1,1,1,1,1,0


In [ ]:
def add_actors(
    nb_synth_clients=0,
    nb_synth_inter=0,
    client_locations=client_locations,
    inter_locations=inter_locations,
    full_time_matrix_car=full_time_matrix_car,
    full_time_matrix_bike=full_time_matrix_bike,
    df_og_needs=c_l_need,
    df_og_inter=df_inter,
):
    nb_og_clients = len(client_locations)
    nb_og_inter = len(inter_locations)
    if nb_synth_inter == 0:
        list_synth_client_loc = sample_random_loc(nb_synth_clients, client_locations)
        new_car_time_matrix, new_bike_time_matrix = add_clients_in_time_matrix(
            list_synth_client_loc,
            full_time_matrix_car,
            full_time_matrix_bike,
            nb_og_clients,
        )

        synth_client_needs_df = df_og_needs[
            ["Prestation", "Durée", "tranche_horaire"]
        ].sample(n=nb_synth_clients, replace=True)
        synth_client_needs_df.reset_index(drop=True, inplace=True)

        synth_inter_df = pd.DataFrame()

    elif nb_synth_clients == 0:
        list_synth_inter_loc = sample_random_loc(nb_synth_inter, inter_locations)
        new_car_time_matrix, new_bike_time_matrix = add_inter_in_time_matrix(
            list_synth_inter_loc,
            full_time_matrix_car,
            full_time_matrix_bike,
            nb_og_clients,
            nb_og_inter,
        )

        synth_client_needs_df = pd.DataFrame()

        synth_inter_df = df_og_inter[
            ["Compétences", "Permis", "Véhicule personnel", "Dispo / Indispo"]
        ].sample(n=nb_synth_inter, replace=True)
        synth_inter_df.reset_index(drop=True, inplace=True)

    else:
        list_synth_client_loc = sample_random_loc(nb_synth_clients, client_locations)
        list_synth_inter_loc = sample_random_loc(nb_synth_inter, inter_locations)
        new_car_time_matrix, new_bike_time_matrix = (
            add_clients_and_inter_in_time_matrix(
                list_synth_client_loc,
                list_synth_inter_loc,
                full_time_matrix_car,
                full_time_matrix_bike,
                nb_og_clients,
                nb_og_inter,
            )
        )

        synth_client_needs_df = df_og_needs[
            ["Prestation", "Durée", "tranche_horaire"]
        ].sample(n=nb_synth_clients, replace=True)
        synth_client_needs_df.reset_index(drop=True, inplace=True)

        synth_inter_df = df_og_inter[
            ["Compétences", "Permis", "Véhicule personnel", "Dispo / Indispo"]
        ].sample(n=nb_synth_inter, replace=True)
        synth_inter_df.reset_index(drop=True, inplace=True)

    return (
        new_car_time_matrix,
        new_bike_time_matrix,
        synth_client_needs_df,
        synth_inter_df,
    )

In [ ]:
def create_real_data_model_with_synth(
    df_og_need,
    full_time_matrix_car,
    full_time_matrix_bike,
    list_available_inter,
    nb_synth_clients,
    nb_synth_inter,
    client_id_to_idx=client_id_to_idx,
    inter_id_to_idx=inter_id_to_idx,
    nb_full_client=nb_full_client,
    list_skills=list_skills,
):
    nb_needs = len(df_og_need)
    nb_inter = len(list_available_inter)
    data = {}

    # Initialization
    data["og_time_matrix_car"] = [
        [4000 for i in range(nb_needs + nb_inter)] for k in range(nb_needs + nb_inter)
    ]
    data["og_time_matrix_bike"] = [
        [4000 for i in range(nb_needs + nb_inter)] for k in range(nb_needs + nb_inter)
    ]
    data["service_times"] = [
        0 for i in range(nb_needs + nb_inter + nb_synth_clients + nb_synth_inter)
    ]
    data["time_windows"] = [
        (0, 1440)
        for i in range(nb_needs + nb_inter + nb_synth_clients + nb_synth_inter)
    ]

    data["num_vehicles"] = nb_inter + nb_synth_inter
    data["is_car"] = [
        1 for i in range(nb_inter + nb_synth_inter)
    ]  # For the moment every inter has a car
    data["starts"] = [
        nb_needs + nb_synth_clients + i for i in range(nb_inter + nb_synth_inter)
    ]
    data["ends"] = [
        nb_needs + nb_synth_clients + i for i in range(nb_inter + nb_synth_inter)
    ]

    data["vehicle_skills"] = [
        list_skills for i in range(nb_inter + nb_synth_inter)
    ]  # For the moment every inter has all skills possible
    data["node_requirements"] = [
        None for i in range(nb_needs + nb_inter + nb_synth_clients + nb_synth_inter)
    ]

    # Create original time matrix
    for i in range(nb_needs):
        need_id_client = df_og_need["ID Client"][i]
        need_idx_client = client_id_to_idx[need_id_client]

        for j in range(nb_needs):
            other_id_client = df_og_need["ID Client"][j]
            other_idx_client = client_id_to_idx[other_id_client]

            travel_time_car = full_time_matrix_car[need_idx_client, other_idx_client]
            travel_time_bike = full_time_matrix_bike[need_idx_client, other_idx_client]
            data["og_time_matrix_car"][i][j] = travel_time_car
            data["og_time_matrix_bike"][i][j] = travel_time_bike

        for j in range(nb_inter):
            id_inter = list_available_inter[j]
            idx_inter = inter_id_to_idx[id_inter]

            travel_time_car = full_time_matrix_car[need_idx_client][
                nb_full_client + idx_inter
            ]
            travel_time_bike = full_time_matrix_bike[need_idx_client][
                nb_full_client + idx_inter
            ]

            data["og_time_matrix_car"][i][nb_needs + j] = travel_time_car
            data["og_time_matrix_car"][nb_needs + j][i] = data["og_time_matrix_car"][i][
                nb_needs + j
            ]
            data["og_time_matrix_bike"][i][nb_needs + j] = travel_time_bike
            data["og_time_matrix_bike"][nb_needs + j][i] = data["og_time_matrix_bike"][
                i
            ][nb_needs + j]
    for i in range(nb_needs + nb_inter):
        data["time_matrix_car"][i][i] = 0
        data["time_matrix_bike"][i][i] = 0

    # Create new time matrix from orginial adding synthetic travel times
    (
        data["time_matrix_car"],
        data["time_matrix_bike"],
        synth_client_needs_df,
        synth_inter_df,
    ) = add_actors(
        nb_synth_clients,
        nb_synth_inter,
        client_locations,
        inter_locations,
        data["og_time_matrix_car"],
        data["og_time_matrix_bike"],
        df_og_need,
        df_inter,
    )

    for i in range(nb_needs + nb_synth_clients):
        if i < nb_needs:
            service_time = int(df_og_need["Durée"][i])
            data["service_times"][i] = service_time

            time_window_temp = (
                df_og_need["tranche_horaire"][i]
                .replace("[", "")
                .replace("]", "")
                .replace(",", "")
                .split()
            )
            time_window_start = int(float(time_window_temp[0]) * 60)
            time_window_end = int(float(time_window_temp[1]) * 60)
            data["time_windows"][i] = (time_window_start, time_window_end)

            skills_required = df_og_need["Prestation"][i]
            data["node_requirements"][i] = skills_required
        else:
            service_time = int(synth_client_needs_df["Durée"][i - nb_needs])
            data["service_times"][i] = service_time

            time_window_temp = (
                synth_client_needs_df["tranche_horaire"][i - nb_needs]
                .replace("[", "")
                .replace("]", "")
                .replace(",", "")
                .split()
            )
            time_window_start = int(float(time_window_temp[0]) * 60)
            time_window_end = int(float(time_window_temp[1]) * 60)
            data["time_windows"][i] = (time_window_start, time_window_end)

            skills_required = synth_client_needs_df["Prestation"][i - nb_needs]
            data["node_requirements"][i] = skills_required

    data["time_windows_with_service"] = [
        (
            data["time_windows"][i][0] + data["service_times"][i],
            data["time_windows"][i][1] + data["service_times"][i],
        )
        for i in range(len(data["service_times"]))
    ]

    data["workload_per_node"] = [1 for i in range(nb_needs + nb_synth_clients)].append(
        [0 for k in range(nb_inter + nb_synth_inter)]
    )  # Useless for now but could be used to differ workload from each node
    data["workload_capacity"] = [5 for i in range(nb_inter + nb_synth_inter)]

    return data

## 6.4. Refined printing result function


In [70]:
def eq_co2_car(travel_time):
    # 10km = 1.9kg CO2
    speed = 0.5  # km/min
    return 1.9 * travel_time * speed / 10


def eq_co2_elec_bike(travel_time):
    # 10km = 20g CO2
    speed = 1 / 3  # km/min
    return 0.02 * travel_time * speed / 10

In [72]:
def print_detailed_solution_real_data(
    data, manager, routing, solution, inter_idx_to_id=inter_idx_to_id
):
    """Prints detailed solution on console, including service and wait times."""
    print(f"Objective: {solution.ObjectiveValue()}")
    time_dimension = routing.GetDimensionOrDie("Time")

    nb_cars_used = 0
    nb_bikes_used = 0
    nb_inter_used = 0

    nb_break_inf = 0
    nb_break_sup = 0
    nb_visits = 0

    avg_length_day = 0
    list_length_day = []

    overall_total_car_commuting_time = 0
    overall_total_bike_commuting_time = 0
    overall_total_commuting_time = 0  # Initialize overall commuting time

    # Display dropped nodes.
    dropped_nodes = "Dropped nodes:"
    for node in range(routing.Size()):
        if routing.IsStart(node) or routing.IsEnd(node):
            continue
        if solution.Value(routing.NextVar(node)) == node:
            dropped_nodes += f" {manager.IndexToNode(node)};"
    print(dropped_nodes)

    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        vehicle_skills = str(data["vehicle_skills"][vehicle_id])
        inter_id = inter_idx_to_id[vehicle_id]
        print(f"Route for vehicle {inter_id} (Skills: {vehicle_skills}):")

        vehicle_commuting_time = 0  # Initialize commuting time for the current vehicle
        nb_steps = 0
        start_day = 0
        end_day = 0
        nb_break_inf_veh = 0
        nb_break_sup_veh = 0

        print(f"Route for vehicle {inter_id}:")
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            next_index = solution.Value(routing.NextVar(index))
            next_node = manager.IndexToNode(next_index)

            if nb_steps == 1:
                if data["is_car"][vehicle_id]:
                    nb_cars_used += 1
                else:
                    nb_bikes_used += 1
                nb_inter_used += 1

            service_time = data["service_times"][manager.IndexToNode(index)]
            skills_required = data["node_requirements"][manager.IndexToNode(index)]
            # Arrival time at current location
            arrival_time = solution.Min(time_var) - service_time

            # End of service time = arrival time + service time at current location
            end_of_service_time = solution.Min(time_var)

            # Initialization
            departure_time = end_of_service_time

            if data["is_car"][vehicle_id]:
                next_travel_time = data["time_matrix_car"][manager.IndexToNode(index)][
                    next_node
                ]
            else:
                next_travel_time = data["time_matrix_bike"][manager.IndexToNode(index)][
                    next_node
                ]
            vehicle_commuting_time += next_travel_time
            if data["is_car"][vehicle_id]:
                overall_total_car_commuting_time += next_travel_time
            else:
                overall_total_bike_commuting_time += next_travel_time
            overall_total_commuting_time += next_travel_time

            if next_index != routing.End(vehicle_id):
                next_arrival_time = solution.Min(time_dimension.CumulVar(next_index))
                next_service_time = data["service_times"][next_node]
                departure_time = (
                    next_arrival_time - next_travel_time - next_service_time
                )

            break_time = departure_time - end_of_service_time

            if nb_steps > 0:
                if break_time > 30:
                    nb_break_sup += 1
                    nb_break_sup_veh += 1
                    nb_visits += 1
                else:
                    if next_index != routing.End(vehicle_id):
                        nb_break_inf += 1
                        nb_break_inf_veh += 1
                        nb_visits += 1

            if nb_steps == 0:
                start_day = departure_time
            if next_index == routing.End(vehicle_id):
                end_day = departure_time

            nb_steps += 1

            if next_index == routing.End(vehicle_id):
                break_time = 0
                departure_time = end_of_service_time

            arrival_time_str = (
                str(arrival_time // 60)
                + "h"
                + (
                    str(arrival_time % 60)
                    if arrival_time % 60 >= 10
                    else "0" + str(arrival_time % 60)
                )
            )
            end_of_service_time_str = (
                str(end_of_service_time // 60)
                + "h"
                + (
                    str(end_of_service_time % 60)
                    if end_of_service_time % 60 >= 10
                    else "0" + str(end_of_service_time % 60)
                )
            )
            break_time_str = (
                str(break_time // 60)
                + "h"
                + (
                    str(break_time % 60)
                    if break_time % 60 >= 10
                    else "0" + str(break_time % 60)
                )
            )
            departure_time_str = (
                str(departure_time // 60)
                + "h"
                + (
                    str(departure_time % 60)
                    if departure_time % 60 >= 10
                    else "0" + str(departure_time % 60)
                )
            )

            # Print detailed timing for current location
            print(
                f"Location {manager.IndexToNode(index) + 1} (Arrival: {arrival_time_str}, "
                f"End of Service Time (Skills required: {skills_required}): {end_of_service_time_str}, Break: {break_time_str}, "
                f"Departure: {departure_time_str}) -> "
            )

            index = next_index

        print(
            f"Number of small breaks (<30min): {nb_break_inf_veh}; Number of big breaks (>30min): {nb_break_sup_veh}"
        )

        # Handle final location's arrival time
        final_arrival_time = solution.Min(time_dimension.CumulVar(index))
        final_arrival_time_str = (
            str(final_arrival_time // 60)
            + "h"
            + (
                str(final_arrival_time % 60)
                if final_arrival_time % 60 >= 10
                else "0" + str(final_arrival_time % 60)
            )
        )
        print(
            f"Location {manager.IndexToNode(index) + 1} (Arrival: {final_arrival_time_str})"
        )
        print(
            f"Total commuting time for vehicle {inter_id}: {vehicle_commuting_time} minutes"
        )

        if end_day - start_day > 1:  # Superior than 1 minute
            list_length_day.append((start_day, end_day))
            avg_length_day += end_day - start_day

            start_day_str = (
                str(start_day // 60)
                + "h"
                + (
                    str(start_day % 60)
                    if start_day % 60 >= 10
                    else "0" + str(start_day % 60)
                )
            )
            end_day_str = (
                str(end_day // 60)
                + "h"
                + (str(end_day % 60) if end_day % 60 >= 10 else "0" + str(end_day % 60))
            )

            print(f"Start of the day: {start_day_str}; End of the day: {end_day_str}")
        print("\n")

    print(f"Overall total commuting time: {overall_total_commuting_time} minutes")
    print(f"Number of cars used: {nb_cars_used}")
    eq_co2_cars = eq_co2_car(overall_total_car_commuting_time)
    print(
        f"Overall total commuting time for cars: {overall_total_car_commuting_time} minutes ({eq_co2_cars} kgCO2)"
    )
    print(f"Number of bikes used: {nb_bikes_used}")
    eq_co2_bikes = eq_co2_elec_bike(overall_total_bike_commuting_time)
    print(
        f"Overall total commuting time for bikes: {overall_total_bike_commuting_time} minutes ({eq_co2_bikes} kgCO2)"
    )

    avg_length_day = avg_length_day / nb_inter_used
    avg_length_day_str = round(avg_length_day)
    avg_length_day_str = (
        str(avg_length_day_str // 60)
        + "h"
        + (
            str(avg_length_day_str % 60)
            if avg_length_day_str % 60 >= 10
            else "0" + str(avg_length_day_str % 60)
        )
    )
    print(f"Average length of a day of work: {avg_length_day_str}")

    dict_results = {
        "total_commuting_time": overall_total_commuting_time,
        "total_car_commuting_time": overall_total_car_commuting_time,
        "total_bike_commuting_time": overall_total_bike_commuting_time,
        "nb_inter_used": nb_inter_used,
        "nb_cars_used": nb_cars_used,
        "nb_bikes_used": nb_bikes_used,
        "avg_length_day": avg_length_day,
        "list_length_day": list_length_day,
        "eq_co2_cars": eq_co2_cars,
        "eq_co2_bikes": eq_co2_bikes,
        "nb_break_inf": nb_break_inf,
        "nb_break_sup": nb_break_sup,
        "nb_visits": nb_visits,
    }

    return dict_results

# 7.
